In [1]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pylab as plt 
import matplotlib.lines as mlines
from matplotlib.legend import Legend
from pythonds.basic.stack import Stack
from math import *
from sklearn.neighbors import KDTree
from lrg_plot_functions import *
from lrg_sum_functions import *
from cosmo_Calc import *
from divideByTwo import *
from read_data import *
from nearNeighbors import *
from scipy import stats
from astropy import stats
import healpy as hp
from localBKG_and_interlopersHEALPix import *
# import healpy as hpy
from makeCutoutTableFunc import *
from plotHistogramProjections3D import *
from read_bricks import *
from mpl_toolkits.axes_grid1 import make_axes_locatable
import random

Notes on what to modify:

Instead of reading in "masked_null" results, read in /Users/mtownsend/Documents/LRG_Data/new_null_<bin>_<zlimit>.npz
These are results where satellites are added *after* masks are applied to null results.
    
Still need to apply masks to real results, as the only saved copies of those I have are unmasked.

In [2]:
# read in models

# lowz bins

low_model25 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_low_25.npz')
low_model25_1 = low_model25['new_low_null_25_1']
low_model25_2 = low_model25['new_low_null_25_2']
low_model25_3 = low_model25['new_low_null_25_3']
low_model25_4 = low_model25['new_low_null_25_4']
low_model25_5 = low_model25['new_low_null_25_5']
low_model25_6 = low_model25['new_low_null_25_6']
low_model25_7 = low_model25['new_low_null_25_7']
low_model25_8 = low_model25['new_low_null_25_8']

low_model03 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_low_03.npz')
low_model03_1 = low_model03['new_low_null_03_1']
low_model03_2 = low_model03['new_low_null_03_2']
low_model03_3 = low_model03['new_low_null_03_3']
low_model03_4 = low_model03['new_low_null_03_4']
low_model03_5 = low_model03['new_low_null_03_5']
low_model03_6 = low_model03['new_low_null_03_6']
low_model03_7 = low_model03['new_low_null_03_7']
low_model03_8 = low_model03['new_low_null_03_8']

In [3]:
# read in models

# mid1 bins

mid1_model25 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid1_25.npz')
mid1_model25_1 = mid1_model25['new_mid1_null_25_1']
mid1_model25_2 = mid1_model25['new_mid1_null_25_2']
mid1_model25_3 = mid1_model25['new_mid1_null_25_3']
mid1_model25_4 = mid1_model25['new_mid1_null_25_4']
mid1_model25_5 = mid1_model25['new_mid1_null_25_5']
mid1_model25_6 = mid1_model25['new_mid1_null_25_6']
mid1_model25_7 = mid1_model25['new_mid1_null_25_7']
mid1_model25_8 = mid1_model25['new_mid1_null_25_8']


mid1_model03 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid1_03.npz')
mid1_model03_1 = mid1_model03['new_mid1_null_03_1']
mid1_model03_2 = mid1_model03['new_mid1_null_03_2']
mid1_model03_3 = mid1_model03['new_mid1_null_03_3']
mid1_model03_4 = mid1_model03['new_mid1_null_03_4']
mid1_model03_5 = mid1_model03['new_mid1_null_03_5']
mid1_model03_6 = mid1_model03['new_mid1_null_03_6']
mid1_model03_7 = mid1_model03['new_mid1_null_03_7']
mid1_model03_8 = mid1_model03['new_mid1_null_03_8']


mid1_model35 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid1_35.npz')
mid1_model35_1 = mid1_model35['new_mid1_null_35_1']
mid1_model35_2 = mid1_model35['new_mid1_null_35_2']
mid1_model35_3 = mid1_model35['new_mid1_null_35_3']
mid1_model35_4 = mid1_model35['new_mid1_null_35_4']
mid1_model35_5 = mid1_model35['new_mid1_null_35_5']
mid1_model35_6 = mid1_model35['new_mid1_null_35_6']
mid1_model35_7 = mid1_model35['new_mid1_null_35_7']
mid1_model35_8 = mid1_model35['new_mid1_null_35_8']


mid1_model04 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid1_04.npz')
mid1_model04_1 = mid1_model04['new_mid1_null_04_1']
mid1_model04_2 = mid1_model04['new_mid1_null_04_2']
mid1_model04_3 = mid1_model04['new_mid1_null_04_3']
mid1_model04_4 = mid1_model04['new_mid1_null_04_4']
mid1_model04_5 = mid1_model04['new_mid1_null_04_5']
mid1_model04_6 = mid1_model04['new_mid1_null_04_6']
mid1_model04_7 = mid1_model04['new_mid1_null_04_7']
mid1_model04_8 = mid1_model04['new_mid1_null_04_8']



In [4]:
# read in models

# mid2 bins

mid2_model25 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid2_25.npz')
mid2_model25_1 = mid2_model25['new_mid2_null_25_1']
mid2_model25_2 = mid2_model25['new_mid2_null_25_2']
mid2_model25_3 = mid2_model25['new_mid2_null_25_3']
mid2_model25_4 = mid2_model25['new_mid2_null_25_4']
mid2_model25_5 = mid2_model25['new_mid2_null_25_5']
mid2_model25_6 = mid2_model25['new_mid2_null_25_6']
mid2_model25_7 = mid2_model25['new_mid2_null_25_7']
mid2_model25_8 = mid2_model25['new_mid2_null_25_8']


mid2_model03 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid2_03.npz')
mid2_model03_1 = mid2_model03['new_mid2_null_03_1']
mid2_model03_2 = mid2_model03['new_mid2_null_03_2']
mid2_model03_3 = mid2_model03['new_mid2_null_03_3']
mid2_model03_4 = mid2_model03['new_mid2_null_03_4']
mid2_model03_5 = mid2_model03['new_mid2_null_03_5']
mid2_model03_6 = mid2_model03['new_mid2_null_03_6']
mid2_model03_7 = mid2_model03['new_mid2_null_03_7']
mid2_model03_8 = mid2_model03['new_mid2_null_03_8']


mid2_model35 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid2_35.npz')
mid2_model35_1 = mid2_model35['new_mid2_null_35_1']
mid2_model35_2 = mid2_model35['new_mid2_null_35_2']
mid2_model35_3 = mid2_model35['new_mid2_null_35_3']
mid2_model35_4 = mid2_model35['new_mid2_null_35_4']
mid2_model35_5 = mid2_model35['new_mid2_null_35_5']
mid2_model35_6 = mid2_model35['new_mid2_null_35_6']
mid2_model35_7 = mid2_model35['new_mid2_null_35_7']
mid2_model35_8 = mid2_model35['new_mid2_null_35_8']


mid2_model04 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid2_04.npz')
mid2_model04_1 = mid2_model04['new_mid2_null_04_1']
mid2_model04_2 = mid2_model04['new_mid2_null_04_2']
mid2_model04_3 = mid2_model04['new_mid2_null_04_3']
mid2_model04_4 = mid2_model04['new_mid2_null_04_4']
mid2_model04_5 = mid2_model04['new_mid2_null_04_5']
mid2_model04_6 = mid2_model04['new_mid2_null_04_6']
mid2_model04_7 = mid2_model04['new_mid2_null_04_7']
mid2_model04_8 = mid2_model04['new_mid2_null_04_8']


mid2_model45 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid2_45.npz')
mid2_model45_1 = mid2_model45['new_mid2_null_45_1']
mid2_model45_2 = mid2_model45['new_mid2_null_45_2']
mid2_model45_3 = mid2_model45['new_mid2_null_45_3']
mid2_model45_4 = mid2_model45['new_mid2_null_45_4']
mid2_model45_5 = mid2_model45['new_mid2_null_45_5']
mid2_model45_6 = mid2_model45['new_mid2_null_45_6']
mid2_model45_7 = mid2_model45['new_mid2_null_45_7']
mid2_model45_8 = mid2_model45['new_mid2_null_45_8']


mid2_model05 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid2_05.npz')
mid2_model05_1 = mid2_model05['new_mid2_null_05_1']
mid2_model05_2 = mid2_model05['new_mid2_null_05_2']
mid2_model05_3 = mid2_model05['new_mid2_null_05_3']
mid2_model05_4 = mid2_model05['new_mid2_null_05_4']
mid2_model05_5 = mid2_model05['new_mid2_null_05_5']
mid2_model05_6 = mid2_model05['new_mid2_null_05_6']
mid2_model05_7 = mid2_model05['new_mid2_null_05_7']
mid2_model05_8 = mid2_model05['new_mid2_null_05_8']


In [5]:
# read in models

# mid3 bins

mid3_model25 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid3_25.npz')
mid3_model25_1 = mid3_model25['new_mid3_null_25_1']
mid3_model25_2 = mid3_model25['new_mid3_null_25_2']
mid3_model25_3 = mid3_model25['new_mid3_null_25_3']
mid3_model25_4 = mid3_model25['new_mid3_null_25_4']
mid3_model25_5 = mid3_model25['new_mid3_null_25_5']
mid3_model25_6 = mid3_model25['new_mid3_null_25_6']
mid3_model25_7 = mid3_model25['new_mid3_null_25_7']
mid3_model25_8 = mid3_model25['new_mid3_null_25_8']


mid3_model03 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid3_03.npz')
mid3_model03_1 = mid3_model03['new_mid3_null_03_1']
mid3_model03_2 = mid3_model03['new_mid3_null_03_2']
mid3_model03_3 = mid3_model03['new_mid3_null_03_3']
mid3_model03_4 = mid3_model03['new_mid3_null_03_4']
mid3_model03_5 = mid3_model03['new_mid3_null_03_5']
mid3_model03_6 = mid3_model03['new_mid3_null_03_6']
mid3_model03_7 = mid3_model03['new_mid3_null_03_7']
mid3_model03_8 = mid3_model03['new_mid3_null_03_8']


mid3_model35 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid3_35.npz')
mid3_model35_1 = mid3_model35['new_mid3_null_35_1']
mid3_model35_2 = mid3_model35['new_mid3_null_35_2']
mid3_model35_3 = mid3_model35['new_mid3_null_35_3']
mid3_model35_4 = mid3_model35['new_mid3_null_35_4']
mid3_model35_5 = mid3_model35['new_mid3_null_35_5']
mid3_model35_6 = mid3_model35['new_mid3_null_35_6']
mid3_model35_7 = mid3_model35['new_mid3_null_35_7']
mid3_model35_8 = mid3_model35['new_mid3_null_35_8']


mid3_model04 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid3_04.npz')
mid3_model04_1 = mid3_model04['new_mid3_null_04_1']
mid3_model04_2 = mid3_model04['new_mid3_null_04_2']
mid3_model04_3 = mid3_model04['new_mid3_null_04_3']
mid3_model04_4 = mid3_model04['new_mid3_null_04_4']
mid3_model04_5 = mid3_model04['new_mid3_null_04_5']
mid3_model04_6 = mid3_model04['new_mid3_null_04_6']
mid3_model04_7 = mid3_model04['new_mid3_null_04_7']
mid3_model04_8 = mid3_model04['new_mid3_null_04_8']


mid3_model45 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid3_45.npz')
mid3_model45_1 = mid3_model45['new_mid3_null_45_1']
mid3_model45_2 = mid3_model45['new_mid3_null_45_2']
mid3_model45_3 = mid3_model45['new_mid3_null_45_3']
mid3_model45_4 = mid3_model45['new_mid3_null_45_4']
mid3_model45_5 = mid3_model45['new_mid3_null_45_5']
mid3_model45_6 = mid3_model45['new_mid3_null_45_6']
mid3_model45_7 = mid3_model45['new_mid3_null_45_7']
mid3_model45_8 = mid3_model45['new_mid3_null_45_8']


mid3_model05 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid3_05.npz')
mid3_model05_1 = mid3_model05['new_mid3_null_05_1']
mid3_model05_2 = mid3_model05['new_mid3_null_05_2']
mid3_model05_3 = mid3_model05['new_mid3_null_05_3']
mid3_model05_4 = mid3_model05['new_mid3_null_05_4']
mid3_model05_5 = mid3_model05['new_mid3_null_05_5']
mid3_model05_6 = mid3_model05['new_mid3_null_05_6']
mid3_model05_7 = mid3_model05['new_mid3_null_05_7']
mid3_model05_8 = mid3_model05['new_mid3_null_05_8']


mid3_model55 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid3_55.npz')
mid3_model55_1 = mid3_model55['new_mid3_null_55_1']
mid3_model55_2 = mid3_model55['new_mid3_null_55_2']
mid3_model55_3 = mid3_model55['new_mid3_null_55_3']
mid3_model55_4 = mid3_model55['new_mid3_null_55_4']
mid3_model55_5 = mid3_model55['new_mid3_null_55_5']
mid3_model55_6 = mid3_model55['new_mid3_null_55_6']
mid3_model55_7 = mid3_model55['new_mid3_null_55_7']
mid3_model55_8 = mid3_model55['new_mid3_null_55_8']


mid3_model06 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_mid3_06.npz')
mid3_model06_1 = mid3_model06['new_mid3_null_06_1']
mid3_model06_2 = mid3_model06['new_mid3_null_06_2']
mid3_model06_3 = mid3_model06['new_mid3_null_06_3']
mid3_model06_4 = mid3_model06['new_mid3_null_06_4']
mid3_model06_5 = mid3_model06['new_mid3_null_06_5']
mid3_model06_6 = mid3_model06['new_mid3_null_06_6']
mid3_model06_7 = mid3_model06['new_mid3_null_06_7']
mid3_model06_8 = mid3_model06['new_mid3_null_06_8']


In [6]:
# read in models

# high bins

high_model25 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_high_25.npz')
high_model25_1 = high_model25['new_high_null_25_1']
high_model25_2 = high_model25['new_high_null_25_2']
high_model25_3 = high_model25['new_high_null_25_3']
high_model25_4 = high_model25['new_high_null_25_4']
high_model25_5 = high_model25['new_high_null_25_5']
high_model25_6 = high_model25['new_high_null_25_6']
high_model25_7 = high_model25['new_high_null_25_7']
high_model25_8 = high_model25['new_high_null_25_8']


high_model03 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_high_03.npz')
high_model03_1 = high_model03['new_high_null_03_1']
high_model03_2 = high_model03['new_high_null_03_2']
high_model03_3 = high_model03['new_high_null_03_3']
high_model03_4 = high_model03['new_high_null_03_4']
high_model03_5 = high_model03['new_high_null_03_5']
high_model03_6 = high_model03['new_high_null_03_6']
high_model03_7 = high_model03['new_high_null_03_7']
high_model03_8 = high_model03['new_high_null_03_8']


high_model35 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_high_35.npz')
high_model35_1 = high_model35['new_high_null_35_1']
high_model35_2 = high_model35['new_high_null_35_2']
high_model35_3 = high_model35['new_high_null_35_3']
high_model35_4 = high_model35['new_high_null_35_4']
high_model35_5 = high_model35['new_high_null_35_5']
high_model35_6 = high_model35['new_high_null_35_6']
high_model35_7 = high_model35['new_high_null_35_7']
high_model35_8 = high_model35['new_high_null_35_8']


high_model04 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_high_04.npz')
high_model04_1 = high_model04['new_high_null_04_1']
high_model04_2 = high_model04['new_high_null_04_2']
high_model04_3 = high_model04['new_high_null_04_3']
high_model04_4 = high_model04['new_high_null_04_4']
high_model04_5 = high_model04['new_high_null_04_5']
high_model04_6 = high_model04['new_high_null_04_6']
high_model04_7 = high_model04['new_high_null_04_7']
high_model04_8 = high_model04['new_high_null_04_8']


high_model45 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_high_45.npz')
high_model45_1 = high_model45['new_high_null_45_1']
high_model45_2 = high_model45['new_high_null_45_2']
high_model45_3 = high_model45['new_high_null_45_3']
high_model45_4 = high_model45['new_high_null_45_4']
high_model45_5 = high_model45['new_high_null_45_5']
high_model45_6 = high_model45['new_high_null_45_6']
high_model45_7 = high_model45['new_high_null_45_7']
high_model45_8 = high_model45['new_high_null_45_8']


high_model05 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_high_05.npz')
high_model05_1 = high_model05['new_high_null_05_1']
high_model05_2 = high_model05['new_high_null_05_2']
high_model05_3 = high_model05['new_high_null_05_3']
high_model05_4 = high_model05['new_high_null_05_4']
high_model05_5 = high_model05['new_high_null_05_5']
high_model05_6 = high_model05['new_high_null_05_6']
high_model05_7 = high_model05['new_high_null_05_7']
high_model05_8 = high_model05['new_high_null_05_8']


high_model55 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_high_55.npz')
high_model55_1 = high_model55['new_high_null_55_1']
high_model55_2 = high_model55['new_high_null_55_2']
high_model55_3 = high_model55['new_high_null_55_3']
high_model55_4 = high_model55['new_high_null_55_4']
high_model55_5 = high_model55['new_high_null_55_5']
high_model55_6 = high_model55['new_high_null_55_6']
high_model55_7 = high_model55['new_high_null_55_7']
high_model55_8 = high_model55['new_high_null_55_8']


high_model06 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_high_06.npz')
high_model06_1 = high_model06['new_high_null_06_1']
high_model06_2 = high_model06['new_high_null_06_2']
high_model06_3 = high_model06['new_high_null_06_3']
high_model06_4 = high_model06['new_high_null_06_4']
high_model06_5 = high_model06['new_high_null_06_5']
high_model06_6 = high_model06['new_high_null_06_6']
high_model06_7 = high_model06['new_high_null_06_7']
high_model06_8 = high_model06['new_high_null_06_8']


high_model65 = np.load('/Users/mtownsend/Documents/LRG_Data/new_null_high_65.npz')
high_model65_1 = high_model65['new_high_null_65_1']
high_model65_2 = high_model65['new_high_null_65_2']
high_model65_3 = high_model65['new_high_null_65_3']
high_model65_4 = high_model65['new_high_null_65_4']
high_model65_5 = high_model65['new_high_null_65_5']
high_model65_6 = high_model65['new_high_null_65_6']
high_model65_7 = high_model65['new_high_null_65_7']
high_model65_8 = high_model65['new_high_null_65_8']



In [7]:
# read in unmasked data results

data_Nsat25 = np.load('/Users/mtownsend/Documents/LRG_Data/data_Nsat25_unmasked.npz')
Nsat_data25 = data_Nsat25['Nsat25']
print(Nsat_data25.shape)

data_Nsat03 = np.load('/Users/mtownsend/Documents/LRG_Data/data_Nsat03_unmasked.npz')
Nsat_data03 = data_Nsat03['Nsat03']
print(Nsat_data03.shape)

data_Nsat35 = np.load('/Users/mtownsend/Documents/LRG_Data/data_Nsat35_unmasked.npz')
Nsat_data35 = data_Nsat35['Nsat35']
print(Nsat_data35.shape)

data_Nsat04 = np.load('/Users/mtownsend/Documents/LRG_Data/data_Nsat04_unmasked.npz')
Nsat_data04 = data_Nsat04['Nsat04']
print(Nsat_data04.shape)

data_Nsat45 = np.load('/Users/mtownsend/Documents/LRG_Data/data_Nsat45_unmasked.npz')
Nsat_data45 = data_Nsat45['Nsat45']
print(Nsat_data45.shape)

data_Nsat05 = np.load('/Users/mtownsend/Documents/LRG_Data/data_Nsat05_unmasked.npz')
Nsat_data05 = data_Nsat05['Nsat05']
print(Nsat_data05.shape)

data_Nsat55 = np.load('/Users/mtownsend/Documents/LRG_Data/data_Nsat55_unmasked.npz')
Nsat_data55 = data_Nsat55['Nsat55']
print(Nsat_data55.shape)

data_Nsat06 = np.load('/Users/mtownsend/Documents/LRG_Data/data_Nsat06_unmasked.npz')
Nsat_data06 = data_Nsat06['Nsat06']
print(Nsat_data06.shape)

data_Nsat65 = np.load('/Users/mtownsend/Documents/LRG_Data/data_Nsat65_unmasked.npz')
Nsat_data65 = data_Nsat65['Nsat65']
print(Nsat_data65.shape)


(90, 49, 49, 49)
(83, 49, 49, 49)
(136, 49, 49, 49)
(128, 49, 49, 49)
(180, 49, 49, 49)
(309, 49, 49, 49)
(386, 49, 49, 49)
(325, 49, 49, 49)
(186, 49, 49, 49)


In [8]:
# read in selection matrices for each luminosity selected sample 

data_smatrix03 = np.load('/Users/mtownsend/Desktop/lowz_smatrix-2matrices.npz')
smatrix03 = data_smatrix03['smatrix_low']
print(smatrix03.shape)

data_smatrix04 = np.load('/Users/mtownsend/Desktop/midz1_smatrix-4matrices.npz')
smatrix04 = data_smatrix04['smatrix_low']
print(smatrix04.shape)

data_smatrix05 = np.load('/Users/mtownsend/Desktop/midz2_smatrix-6matrices.npz')
smatrix05 = data_smatrix05['smatrix_low']
print(smatrix05.shape)

data_smatrix06 = np.load('/Users/mtownsend/Desktop/midz3_smatrix-8matrices.npz')
smatrix06 = data_smatrix06['smatrix_low']
print(smatrix06.shape)

data_smatrix65 = np.load('/Users/mtownsend/Desktop/highz_smatrix-9matrices.npz')
smatrix65 = data_smatrix65['smatrix_low']
print(smatrix65.shape)

(2, 49, 49, 49)
(4, 49, 49, 49)
(6, 49, 49, 49)
(8, 49, 49, 49)
(9, 49, 49, 49)


In [9]:
# make masked Nsat for log(lum)=9.55; 0.2 < z < 0.3

# 0.2 < z < 0.25
lowz_maskedNsat25 = []
for i in range(len(Nsat_data25)):
    lowz_maskedNsat25.append(Nsat_data25[i] * smatrix03[0])
    
# 0.25 < z < 0.3
lowz_maskedNsat03 = []
for i in range(len(Nsat_data03)):
    lowz_maskedNsat03.append(Nsat_data03[i] * smatrix03[1])

print(np.asarray(lowz_maskedNsat25).shape)
print(np.asarray(lowz_maskedNsat03).shape)

(90, 49, 49, 49)
(83, 49, 49, 49)


In [10]:
# make masked Nsat for log(lum)=9.84; 0.2 < z < 0.4

# 0.2 < z < 0.25
mid1_maskedNsat25 = []
for i in range(len(Nsat_data25)):
    mid1_maskedNsat25.append(Nsat_data25[i] * smatrix04[0])
    
# 0.25 < z < 0.3
mid1_maskedNsat03 = []
for i in range(len(Nsat_data03)):
    mid1_maskedNsat03.append(Nsat_data03[i] * smatrix04[1])
    
# 0.3 < z < 0.35
mid1_maskedNsat35 = []
for i in range(len(Nsat_data35)):
    mid1_maskedNsat35.append(Nsat_data35[i] * smatrix04[2])
    
# 0.35 < z < 0.4
mid1_maskedNsat04 = []
for i in range(len(Nsat_data04)):
    mid1_maskedNsat04.append(Nsat_data04[i] * smatrix04[3])


print(np.asarray(mid1_maskedNsat25).shape)
print(np.asarray(mid1_maskedNsat03).shape)
print(np.asarray(mid1_maskedNsat35).shape)
print(np.asarray(mid1_maskedNsat04).shape)

(90, 49, 49, 49)
(83, 49, 49, 49)
(136, 49, 49, 49)
(128, 49, 49, 49)


In [11]:
# make masked Nsat for log(lum)=10.13; 0.2 < z < 0.5

# 0.2 < z < 0.25
mid2_maskedNsat25 = []
for i in range(len(Nsat_data25)):
    mid2_maskedNsat25.append(Nsat_data25[i] * smatrix05[0])
    
# 0.25 < z < 0.3
mid2_maskedNsat03 = []
for i in range(len(Nsat_data03)):
    mid2_maskedNsat03.append(Nsat_data03[i] * smatrix05[1])
    
# 0.3 < z < 0.35
mid2_maskedNsat35 = []
for i in range(len(Nsat_data35)):
    mid2_maskedNsat35.append(Nsat_data35[i] * smatrix05[2])
    
# 0.35 < z < 0.4
mid2_maskedNsat04 = []
for i in range(len(Nsat_data04)):
    mid2_maskedNsat04.append(Nsat_data04[i] * smatrix05[3])
    
# 0.4 < z < 0.45
mid2_maskedNsat45 = []
for i in range(len(Nsat_data45)):
    mid2_maskedNsat45.append(Nsat_data45[i] * smatrix05[4])
    
# 0.45 < z < 0.5
mid2_maskedNsat05 = []
for i in range(len(Nsat_data05)):
    mid2_maskedNsat05.append(Nsat_data05[i] * smatrix05[5])


print(np.asarray(mid2_maskedNsat25).shape)
print(np.asarray(mid2_maskedNsat03).shape)
print(np.asarray(mid2_maskedNsat35).shape)
print(np.asarray(mid2_maskedNsat04).shape)
print(np.asarray(mid2_maskedNsat45).shape)
print(np.asarray(mid2_maskedNsat05).shape)

(90, 49, 49, 49)
(83, 49, 49, 49)
(136, 49, 49, 49)
(128, 49, 49, 49)
(180, 49, 49, 49)
(309, 49, 49, 49)


In [12]:
# make masked Nsat for log(lum)=10.36; 0.2 < z < 0.6

# 0.2 < z < 0.25
mid3_maskedNsat25 = []
for i in range(len(Nsat_data25)):
    mid3_maskedNsat25.append(Nsat_data25[i] * smatrix06[0])
    
# 0.25 < z < 0.3
mid3_maskedNsat03 = []
for i in range(len(Nsat_data03)):
    mid3_maskedNsat03.append(Nsat_data03[i] * smatrix06[1])
    
# 0.3 < z < 0.35
mid3_maskedNsat35 = []
for i in range(len(Nsat_data35)):
    mid3_maskedNsat35.append(Nsat_data35[i] * smatrix06[2])
    
# 0.35 < z < 0.4
mid3_maskedNsat04 = []
for i in range(len(Nsat_data04)):
    mid3_maskedNsat04.append(Nsat_data04[i] * smatrix06[3])
    
# 0.4 < z < 0.45
mid3_maskedNsat45 = []
for i in range(len(Nsat_data45)):
    mid3_maskedNsat45.append(Nsat_data45[i] * smatrix06[4])
    
# 0.45 < z < 0.5
mid3_maskedNsat05 = []
for i in range(len(Nsat_data05)):
    mid3_maskedNsat05.append(Nsat_data05[i] * smatrix06[5])
    
# 0.5 < z < 0.55
mid3_maskedNsat55 = []
for i in range(len(Nsat_data55)):
    mid3_maskedNsat55.append(Nsat_data55[i] * smatrix06[6])
    
# 0.55 < z < 0.6
mid3_maskedNsat06 = []
for i in range(len(Nsat_data06)):
    mid3_maskedNsat06.append(Nsat_data06[i] * smatrix06[7])


print(np.asarray(mid3_maskedNsat25).shape)
print(np.asarray(mid3_maskedNsat03).shape)
print(np.asarray(mid3_maskedNsat35).shape)
print(np.asarray(mid3_maskedNsat04).shape)
print(np.asarray(mid3_maskedNsat45).shape)
print(np.asarray(mid3_maskedNsat05).shape)
print(np.asarray(mid3_maskedNsat55).shape)
print(np.asarray(mid3_maskedNsat06).shape)

(90, 49, 49, 49)
(83, 49, 49, 49)
(136, 49, 49, 49)
(128, 49, 49, 49)
(180, 49, 49, 49)
(309, 49, 49, 49)
(386, 49, 49, 49)
(325, 49, 49, 49)


In [13]:
# make masked Nsat for log(lum)=10.47; 0.2 < z < 0.65

# 0.2 < z < 0.25
high_maskedNsat25 = []
for i in range(len(Nsat_data25)):
    high_maskedNsat25.append(Nsat_data25[i] * smatrix65[0])
    
# 0.25 < z < 0.3
high_maskedNsat03 = []
for i in range(len(Nsat_data03)):
    high_maskedNsat03.append(Nsat_data03[i] * smatrix65[1])
    
# 0.3 < z < 0.35
high_maskedNsat35 = []
for i in range(len(Nsat_data35)):
    high_maskedNsat35.append(Nsat_data35[i] * smatrix65[2])
    
# 0.35 < z < 0.4
high_maskedNsat04 = []
for i in range(len(Nsat_data04)):
    high_maskedNsat04.append(Nsat_data04[i] * smatrix65[3])
    
# 0.4 < z < 0.45
high_maskedNsat45 = []
for i in range(len(Nsat_data45)):
    high_maskedNsat45.append(Nsat_data45[i] * smatrix65[4])
    
# 0.45 < z < 0.5
high_maskedNsat05 = []
for i in range(len(Nsat_data05)):
    high_maskedNsat05.append(Nsat_data05[i] * smatrix65[5])
    
# 0.5 < z < 0.55
high_maskedNsat55 = []
for i in range(len(Nsat_data55)):
    high_maskedNsat55.append(Nsat_data55[i] * smatrix65[6])
    
# 0.55 < z < 0.6
high_maskedNsat06 = []
for i in range(len(Nsat_data06)):
    high_maskedNsat06.append(Nsat_data06[i] * smatrix65[7])

# 0.6 < z < 0.65
high_maskedNsat65 = []
for i in range(len(Nsat_data65)):
    high_maskedNsat65.append(Nsat_data65[i] * smatrix65[8])


print(np.asarray(high_maskedNsat25).shape)
print(np.asarray(high_maskedNsat03).shape)
print(np.asarray(high_maskedNsat35).shape)
print(np.asarray(high_maskedNsat04).shape)
print(np.asarray(high_maskedNsat45).shape)
print(np.asarray(high_maskedNsat05).shape)
print(np.asarray(high_maskedNsat55).shape)
print(np.asarray(high_maskedNsat06).shape)
print(np.asarray(high_maskedNsat65).shape)

(90, 49, 49, 49)
(83, 49, 49, 49)
(136, 49, 49, 49)
(128, 49, 49, 49)
(180, 49, 49, 49)
(309, 49, 49, 49)
(386, 49, 49, 49)
(325, 49, 49, 49)
(186, 49, 49, 49)


In [20]:
def totalNsat(Nsat, lum, z1, z2, string):

    import matplotlib.pylab as plt 
    import numpy as np
    from scipy import stats
    
    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height


    sumsat = []

    # Sum up number of satellite galaxies for every LRG
    for i in range(len(Nsat)):
        sumsat.append(np.sum(Nsat[i]))

    meansat = np.mean(sumsat)
    print("mean number of satellites is", meansat)
    
    mediansat = np.median(sumsat)
#     print("median number of satellites is", mediansat)

    sdsat = np.std(sumsat)
#     print("standard deviation of satellites is", sdsat)

    sterr = stats.sem(sumsat)
#     print("standard error is", sterr)
    
    eighty = np.percentile(sumsat, 80.)
#     print("80th percentile = ", eighty)
    ninety = np.percentile(sumsat, 90.)
#     print("90th percentile = ", ninety)
    ninetynine = np.percentile(sumsat, 99.)
#     print("99th percentile = ", ninetynine)

#     plt.rcParams["figure.figsize"] = [15, 8]
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
# #     plt.title("Histogram of the Number of Satellite Galaxies; log(lum)=10.36; 0.55 < z < 0.6", fontsize=25)
#     plt.title("Histogram of the Number of Satellite Galaxies; log(lum)={}; {} < z < {}; ({})".format(lum, z1, z2, string), fontsize=18)
#     plt.hist(sumsat, bins=50)
#     plt.xlabel(r'$Number$ $of$ $Satellite$ $Galaxies$', fontsize=20)
#     plt.ylabel(r'$counts$', fontsize=20)
# #     plt.axvline(x=eighty, linewidth=1, ls='-', color='r')
# #     plt.axvline(x=ninety, linewidth=1, ls='--', color='g')
# #     plt.axvline(x=ninetynine, linewidth=1, ls=':', color='b')
#     plt.text(top, right,'mean = {}'.format(meansat), fontsize=14, ha='center', va='center', transform=ax.transAxes)
#     plt.text(top, right-0.05,'median = {}'.format(mediansat), fontsize=14, ha='center', va='center', transform=ax.transAxes)
# #     plt.text(top, right-0.1,'99th = {}'.format(ninetynine), fontsize=14, ha='center', va='center', transform=ax.transAxes)
#     # plt.show()
    return(sumsat)

In [15]:
# misc data for use in A-D function

rz = np.load('/Users/mtownsend/Documents/LRG_Data/random_rzcolor.npz')

rz_rand1 = rz['rz_rand1']
print(len(rz_rand1))
rz_rand2 = rz['rz_rand2']
print(len(rz_rand2))
rz_rand3 = rz['rz_rand3']
print(len(rz_rand3))
rz_rand4 = rz['rz_rand4']
print(len(rz_rand4))
rz_rand5 = rz['rz_rand5']
print(len(rz_rand5))
rz_rand6 = rz['rz_rand6']
print(len(rz_rand6))
rz_rand7 = rz['rz_rand7']
print(len(rz_rand7))
rz_rand8 = rz['rz_rand8']
print(len(rz_rand8))

# create redshift bins

zinit = 0.2
zfin = 0.65
interval = 0.05
# num = (zfin-zinit)/interval
num = 10. 
zrange, step = np.linspace(zinit, zfin, num=int(num), retstep=True)


3
1
4
4
5
5
7
7


In [42]:
# model comparision for lowz 

# models for 0.2 < 0.25

sumsat25_model1 = totalNsat(low_model25_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat25_model2 = totalNsat(low_model25_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat25_model3 = totalNsat(low_model25_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat25_model4 = totalNsat(low_model25_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat25_model5 = totalNsat(low_model25_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat25_model6 = totalNsat(low_model25_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat25_model7 = totalNsat(low_model25_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat25_model8 = totalNsat(low_model25_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')


# models for 0.25 < 0.3

sumsat03_model1 = totalNsat(low_model03_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat03_model2 = totalNsat(low_model03_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat03_model3 = totalNsat(low_model03_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat03_model4 = totalNsat(low_model03_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat03_model5 = totalNsat(low_model03_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat03_model6 = totalNsat(low_model03_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat03_model7 = totalNsat(low_model03_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat03_model8 = totalNsat(low_model03_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')


# data

sumsat25_data = totalNsat(lowz_maskedNsat25, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat03_data = totalNsat(lowz_maskedNsat03, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')


mean number of satellites is 3.139612875886785
mean number of satellites is 1.1396128758867852
mean number of satellites is 4.139612875886785
mean number of satellites is 4.139612875886786
mean number of satellites is 5.139612875886785
mean number of satellites is 5.139612875886785
mean number of satellites is 7.139612875886785
mean number of satellites is 7.139612875886785
mean number of satellites is 3.2643416258102373
mean number of satellites is 1.2643416258102371
mean number of satellites is 4.264341625810237
mean number of satellites is 4.264341625810237
mean number of satellites is 5.264341625810237
mean number of satellites is 5.264341625810237
mean number of satellites is 7.2643416258102365
mean number of satellites is 7.2643416258102365
mean number of satellites is 0.732166005879621
mean number of satellites is 0.6223153506671162


In [22]:
# perform k-sample Anderson-Darling test on model distributions and data

from scipy import stats

stat25_1, crit_vals25_1, sig_level25_1 = stats.anderson_ksamp([sumsat25_model1, sumsat25_data])
stat25_2, crit_vals25_2, sig_level25_2 = stats.anderson_ksamp([sumsat25_model2, sumsat25_data])
stat25_3, crit_vals25_3, sig_level25_3 = stats.anderson_ksamp([sumsat25_model3, sumsat25_data])
stat25_4, crit_vals25_4, sig_level25_4 = stats.anderson_ksamp([sumsat25_model4, sumsat25_data])
stat25_5, crit_vals25_5, sig_level25_5 = stats.anderson_ksamp([sumsat25_model5, sumsat25_data])
stat25_6, crit_vals25_6, sig_level25_6 = stats.anderson_ksamp([sumsat25_model6, sumsat25_data])
stat25_7, crit_vals25_7, sig_level25_7 = stats.anderson_ksamp([sumsat25_model7, sumsat25_data])
stat25_8, crit_vals25_8, sig_level25_8 = stats.anderson_ksamp([sumsat25_model8, sumsat25_data])

stat03_1, crit_vals03_1, sig_level03_1 = stats.anderson_ksamp([sumsat03_model1, sumsat03_data])
stat03_2, crit_vals03_2, sig_level03_2 = stats.anderson_ksamp([sumsat03_model2, sumsat03_data])
stat03_3, crit_vals03_3, sig_level03_3 = stats.anderson_ksamp([sumsat03_model3, sumsat03_data])
stat03_4, crit_vals03_4, sig_level03_4 = stats.anderson_ksamp([sumsat03_model4, sumsat03_data])
stat03_5, crit_vals03_5, sig_level03_5 = stats.anderson_ksamp([sumsat03_model5, sumsat03_data])
stat03_6, crit_vals03_6, sig_level03_6 = stats.anderson_ksamp([sumsat03_model6, sumsat03_data])
stat03_7, crit_vals03_7, sig_level03_7 = stats.anderson_ksamp([sumsat03_model7, sumsat03_data])
stat03_8, crit_vals03_8, sig_level03_8 = stats.anderson_ksamp([sumsat03_model8, sumsat03_data])

print('0.2 < 0.25')
print('----')
print('model 1')
print(stat25_1)
print(crit_vals25_1)
print(sig_level25_1)
print('----')

print('model 2')
print(stat25_2)
print(crit_vals25_2)
print(sig_level25_2)
print('----')

print('model 3')
print(stat25_3)
print(crit_vals25_3)
print(sig_level25_3)
print('----')

print('model 4')
print(stat25_4)
print(crit_vals25_4)
print(sig_level25_4)
print('----')

print('model 5')
print(stat25_5)
print(crit_vals25_5)
print(sig_level25_5)
print('----')

print('model 6')
print(stat25_6)
print(crit_vals25_6)
print(sig_level25_6)
print('----')

print('model 7')
print(stat25_7)
print(crit_vals25_7)
print(sig_level25_7)
print('----')

print('model 8')
print(stat25_8)
print(crit_vals25_8)
print(sig_level25_8)
print('----')
print()
print('----')
print('----')
print()
print('0.25 < 0.3')
print('----')
print('model 1')
print(stat03_1)
print(crit_vals03_1)
print(sig_level03_1)
print('----')

print('model 2')
print(stat03_2)
print(crit_vals03_2)
print(sig_level03_2)
print('----')

print('model 3')
print(stat03_3)
print(crit_vals03_3)
print(sig_level03_3)
print('----')

print('model 4')
print(stat03_4)
print(crit_vals03_4)
print(sig_level03_4)
print('----')

print('model 5')
print(stat03_5)
print(crit_vals03_5)
print(sig_level03_5)
print('----')

print('model 6')
print(stat03_6)
print(crit_vals03_6)
print(sig_level03_6)
print('----')

print('model 7')
print(stat03_7)
print(crit_vals03_7)
print(sig_level03_7)
print('----')

print('model 8')
print(stat03_8)
print(crit_vals03_8)
print(sig_level03_8)



0.2 < 0.25
----
model 1
230.38183949623226
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 2
34.92353966183831
[0.325 1.226 1.961 2.718 3.752]
0.0014553787059856807
----
model 3
280.14752518018804
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 4
280.14752518018804
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 5
303.0951355179849
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 6
303.0951355179849
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 7
351.07751975006425
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 8
351.07751975006425
[0.325 1.226 1.961 2.718 3.752]
inf
----

----
----

0.25 < 0.3
----
model 1
164.6431904781417
[0.325 1.226 1.961 2.718 3.752]
8.974034443282265e+222
----
model 2
10.257321666427979
[0.325 1.226 1.961 2.718 3.752]
0.00011577257881460882
----
model 3
247.9568853252105
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 4
247.9568853252105
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 5
283.43720558870825
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 6
283.4372055887

In [43]:
# perform k-sample K-S test on model distributions and data

from scipy import stats

stat25_1, pval25_1 = stats.ks_2samp(sumsat25_model1, sumsat25_data)
stat25_2, pval25_2 = stats.ks_2samp(sumsat25_model2, sumsat25_data)
stat25_3, pval25_3 = stats.ks_2samp(sumsat25_model3, sumsat25_data)
stat25_4, pval25_4 = stats.ks_2samp(sumsat25_model4, sumsat25_data)
stat25_5, pval25_5 = stats.ks_2samp(sumsat25_model5, sumsat25_data)
stat25_6, pval25_6 = stats.ks_2samp(sumsat25_model6, sumsat25_data)
stat25_7, pval25_7 = stats.ks_2samp(sumsat25_model7, sumsat25_data)
stat25_8, pval25_8 = stats.ks_2samp(sumsat25_model8, sumsat25_data)

stat03_1, pval03_1 = stats.ks_2samp(sumsat03_model1, sumsat03_data)
stat03_2, pval03_2 = stats.ks_2samp(sumsat03_model2, sumsat03_data)
stat03_3, pval03_3 = stats.ks_2samp(sumsat03_model3, sumsat03_data)
stat03_4, pval03_4 = stats.ks_2samp(sumsat03_model4, sumsat03_data)
stat03_5, pval03_5 = stats.ks_2samp(sumsat03_model5, sumsat03_data)
stat03_6, pval03_6 = stats.ks_2samp(sumsat03_model6, sumsat03_data)
stat03_7, pval03_7 = stats.ks_2samp(sumsat03_model7, sumsat03_data)
stat03_8, pval03_8 = stats.ks_2samp(sumsat03_model8, sumsat03_data)

print('0.2 < 0.25')
print('----')
print('model 1')
print(stat25_1)
print(pval25_1)
print('----')

print('model 2')
print(stat25_2)
print(pval25_2)
print('----')

print('model 3')
print(stat25_3)
print(pval25_3)
print('----')

print('model 4')
print(stat25_4)
print(pval25_4)
print('----')

print('model 5')
print(stat25_5)
print(pval25_5)
print('----')

print('model 6')
print(stat25_6)
print(pval25_6)
print('----')

print('model 7')
print(stat25_7)
print(pval25_7)
print('----')

print('model 8')
print(stat25_8)
print(pval25_8)
print('----')
print()
print('----')
print('----')
print()
print('0.25 < 0.3')
print('----')
print('model 1')
print(stat03_1)
print(pval03_1)
print('----')

print('model 2')
print(stat03_2)
print(pval03_2)
print('----')

print('model 3')
print(stat03_3)
print(pval03_3)
print('----')

print('model 4')
print(stat03_4)
print(pval03_4)
print('----')

print('model 5')
print(stat03_5)
print(pval03_5)
print('----')

print('model 6')
print(stat03_6)
print(pval03_6)
print('----')

print('model 7')
print(stat03_7)
print(pval03_7)
print('----')

print('model 8')
print(stat03_8)
print(pval03_8)



0.2 < 0.25
----
model 1
0.7726319499515375
9.113265801001786e-47
----
model 2
0.3000088113490175
2.0607197091849655e-07
----
model 3
0.8801656533615296
1.454045411290282e-60
----
model 4
0.8801656533615296
1.454045411290282e-60
----
model 5
0.9075425147590096
2.3082192161514235e-64
----
model 6
0.9075425147590096
2.3082192161514235e-64
----
model 7
0.9920697858842189
7.91551924648637e-77
----
model 8
0.9920697858842189
7.91551924648637e-77
----

----
----

0.25 < 0.3
----
model 1
0.6399836634674291
7.596600828011003e-30
----
model 2
0.2174898139774595
0.0008003879024095427
----
model 3
0.8259624454233399
1.9816503458985085e-49
----
model 4
0.8259624454233399
1.9816503458985085e-49
----
model 5
0.8967540865641744
3.4432897142590894e-58
----
model 6
0.8967540865641744
3.4432897142590894e-58
----
model 7
0.9714645506967337
3.2445111206482256e-68
----
model 8
0.9714645506967337
3.2445111206482256e-68


In [44]:
# model comparision for mid1 

# models for 0.2 < 0.25

sumsat25_model1 = totalNsat(mid1_model25_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat25_model2 = totalNsat(mid1_model25_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat25_model3 = totalNsat(mid1_model25_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat25_model4 = totalNsat(mid1_model25_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat25_model5 = totalNsat(mid1_model25_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat25_model6 = totalNsat(mid1_model25_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat25_model7 = totalNsat(mid1_model25_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat25_model8 = totalNsat(mid1_model25_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.25 < 0.3

sumsat03_model1 = totalNsat(mid1_model03_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat03_model2 = totalNsat(mid1_model03_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat03_model3 = totalNsat(mid1_model03_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat03_model4 = totalNsat(mid1_model03_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat03_model5 = totalNsat(mid1_model03_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat03_model6 = totalNsat(mid1_model03_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat03_model7 = totalNsat(mid1_model03_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat03_model8 = totalNsat(mid1_model03_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.3 < 0.35

sumsat35_model1 = totalNsat(mid1_model35_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat35_model2 = totalNsat(mid1_model35_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat35_model3 = totalNsat(mid1_model35_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat35_model4 = totalNsat(mid1_model35_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat35_model5 = totalNsat(mid1_model35_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat35_model6 = totalNsat(mid1_model35_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat35_model7 = totalNsat(mid1_model35_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat35_model8 = totalNsat(mid1_model35_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.35 < 0.4

sumsat04_model1 = totalNsat(mid1_model04_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat04_model2 = totalNsat(mid1_model04_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat04_model3 = totalNsat(mid1_model04_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat04_model4 = totalNsat(mid1_model04_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat04_model5 = totalNsat(mid1_model04_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat04_model6 = totalNsat(mid1_model04_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat04_model7 = totalNsat(mid1_model04_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat04_model8 = totalNsat(mid1_model04_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# data

sumsat25_data = totalNsat(mid1_maskedNsat25, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat03_data = totalNsat(mid1_maskedNsat03, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat35_data = totalNsat(mid1_maskedNsat35, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat04_data = totalNsat(mid1_maskedNsat04, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')


mean number of satellites is 3.0823496239915937
mean number of satellites is 1.0823496239915942
mean number of satellites is 4.082349623991595
mean number of satellites is 4.082349623991595
mean number of satellites is 5.082349623991594
mean number of satellites is 5.082349623991595
mean number of satellites is 7.082349623991593
mean number of satellites is 7.082349623991593
----
mean number of satellites is 3.0939149297954422
mean number of satellites is 1.0939149297954418
mean number of satellites is 4.093914929795441
mean number of satellites is 4.093914929795441
mean number of satellites is 5.093914929795441
mean number of satellites is 5.093914929795441
mean number of satellites is 7.093914929795441
mean number of satellites is 7.093914929795441
----
mean number of satellites is 3.234059768612343
mean number of satellites is 1.234059768612343
mean number of satellites is 4.234059768612343
mean number of satellites is 4.234059768612343
mean number of satellites is 5.234059768612343

In [32]:
# perform k-sample Anderson-Darling test on model distributions and data

from scipy import stats

stat25_1, crit_vals25_1, sig_level25_1 = stats.anderson_ksamp([sumsat25_model1, sumsat25_data])
stat25_2, crit_vals25_2, sig_level25_2 = stats.anderson_ksamp([sumsat25_model2, sumsat25_data])
stat25_3, crit_vals25_3, sig_level25_3 = stats.anderson_ksamp([sumsat25_model3, sumsat25_data])
stat25_4, crit_vals25_4, sig_level25_4 = stats.anderson_ksamp([sumsat25_model4, sumsat25_data])
stat25_5, crit_vals25_5, sig_level25_5 = stats.anderson_ksamp([sumsat25_model5, sumsat25_data])
stat25_6, crit_vals25_6, sig_level25_6 = stats.anderson_ksamp([sumsat25_model6, sumsat25_data])
stat25_7, crit_vals25_7, sig_level25_7 = stats.anderson_ksamp([sumsat25_model7, sumsat25_data])
stat25_8, crit_vals25_8, sig_level25_8 = stats.anderson_ksamp([sumsat25_model8, sumsat25_data])

stat03_1, crit_vals03_1, sig_level03_1 = stats.anderson_ksamp([sumsat03_model1, sumsat03_data])
stat03_2, crit_vals03_2, sig_level03_2 = stats.anderson_ksamp([sumsat03_model2, sumsat03_data])
stat03_3, crit_vals03_3, sig_level03_3 = stats.anderson_ksamp([sumsat03_model3, sumsat03_data])
stat03_4, crit_vals03_4, sig_level03_4 = stats.anderson_ksamp([sumsat03_model4, sumsat03_data])
stat03_5, crit_vals03_5, sig_level03_5 = stats.anderson_ksamp([sumsat03_model5, sumsat03_data])
stat03_6, crit_vals03_6, sig_level03_6 = stats.anderson_ksamp([sumsat03_model6, sumsat03_data])
stat03_7, crit_vals03_7, sig_level03_7 = stats.anderson_ksamp([sumsat03_model7, sumsat03_data])
stat03_8, crit_vals03_8, sig_level03_8 = stats.anderson_ksamp([sumsat03_model8, sumsat03_data])

stat35_1, crit_vals35_1, sig_level35_1 = stats.anderson_ksamp([sumsat35_model1, sumsat35_data])
stat35_2, crit_vals35_2, sig_level35_2 = stats.anderson_ksamp([sumsat35_model2, sumsat35_data])
stat35_3, crit_vals35_3, sig_level35_3 = stats.anderson_ksamp([sumsat35_model3, sumsat35_data])
stat35_4, crit_vals35_4, sig_level35_4 = stats.anderson_ksamp([sumsat35_model4, sumsat35_data])
stat35_5, crit_vals35_5, sig_level35_5 = stats.anderson_ksamp([sumsat35_model5, sumsat35_data])
stat35_6, crit_vals35_6, sig_level35_6 = stats.anderson_ksamp([sumsat35_model6, sumsat35_data])
stat35_7, crit_vals35_7, sig_level35_7 = stats.anderson_ksamp([sumsat35_model7, sumsat35_data])
stat35_8, crit_vals35_8, sig_level35_8 = stats.anderson_ksamp([sumsat35_model8, sumsat35_data])

stat04_1, crit_vals04_1, sig_level04_1 = stats.anderson_ksamp([sumsat04_model1, sumsat04_data])
stat04_2, crit_vals04_2, sig_level04_2 = stats.anderson_ksamp([sumsat04_model2, sumsat04_data])
stat04_3, crit_vals04_3, sig_level04_3 = stats.anderson_ksamp([sumsat04_model3, sumsat04_data])
stat04_4, crit_vals04_4, sig_level04_4 = stats.anderson_ksamp([sumsat04_model4, sumsat04_data])
stat04_5, crit_vals04_5, sig_level04_5 = stats.anderson_ksamp([sumsat04_model5, sumsat04_data])
stat04_6, crit_vals04_6, sig_level04_6 = stats.anderson_ksamp([sumsat04_model6, sumsat04_data])
stat04_7, crit_vals04_7, sig_level04_7 = stats.anderson_ksamp([sumsat04_model7, sumsat04_data])
stat04_8, crit_vals04_8, sig_level04_8 = stats.anderson_ksamp([sumsat04_model8, sumsat04_data])

print('0.2 < 0.25')
print('----')
print('model 1')
print(stat25_1)
print(crit_vals25_1)
print(sig_level25_1)
print('----')

print('model 2')
print(stat25_2)
print(crit_vals25_2)
print(sig_level25_2)
print('----')

print('model 3')
print(stat25_3)
print(crit_vals25_3)
print(sig_level25_3)
print('----')

print('model 4')
print(stat25_4)
print(crit_vals25_4)
print(sig_level25_4)
print('----')

print('model 5')
print(stat25_5)
print(crit_vals25_5)
print(sig_level25_5)
print('----')

print('model 6')
print(stat25_6)
print(crit_vals25_6)
print(sig_level25_6)
print('----')

print('model 7')
print(stat25_7)
print(crit_vals25_7)
print(sig_level25_7)
print('----')

print('model 8')
print(stat25_8)
print(crit_vals25_8)
print(sig_level25_8)
print('----')
print()
print('----')
print('----')
print()
print('0.25 < 0.3')
print('----')
print('model 1')
print(stat03_1)
print(crit_vals03_1)
print(sig_level03_1)
print('----')

print('model 2')
print(stat03_2)
print(crit_vals03_2)
print(sig_level03_2)
print('----')

print('model 3')
print(stat03_3)
print(crit_vals03_3)
print(sig_level03_3)
print('----')

print('model 4')
print(stat03_4)
print(crit_vals03_4)
print(sig_level03_4)
print('----')

print('model 5')
print(stat03_5)
print(crit_vals03_5)
print(sig_level03_5)
print('----')

print('model 6')
print(stat03_6)
print(crit_vals03_6)
print(sig_level03_6)
print('----')

print('model 7')
print(stat03_7)
print(crit_vals03_7)
print(sig_level03_7)
print('----')

print('model 8')
print(stat03_8)
print(crit_vals03_8)
print(sig_level03_8)
print('----')
print()
print('----')
print('----')
print()
print('0.3 < 0.35')
print('----')
print('model 1')
print(stat35_1)
print(crit_vals35_1)
print(sig_level35_1)
print('----')

print('model 2')
print(stat35_2)
print(crit_vals35_2)
print(sig_level35_2)
print('----')

print('model 3')
print(stat35_3)
print(crit_vals35_3)
print(sig_level35_3)
print('----')

print('model 4')
print(stat35_4)
print(crit_vals35_4)
print(sig_level35_4)
print('----')

print('model 5')
print(stat35_5)
print(crit_vals35_5)
print(sig_level35_5)
print('----')

print('model 6')
print(stat35_6)
print(crit_vals35_6)
print(sig_level35_6)
print('----')

print('model 7')
print(stat35_7)
print(crit_vals35_7)
print(sig_level35_7)
print('----')

print('model 8')
print(stat35_8)
print(crit_vals35_8)
print(sig_level35_8)
print('----')
print()
print('----')
print('----')
print()
print('0.35 < 0.4')
print('----')
print('model 1')
print(stat04_1)
print(crit_vals04_1)
print(sig_level04_1)
print('----')

print('model 2')
print(stat04_2)
print(crit_vals04_2)
print(sig_level04_2)
print('----')

print('model 3')
print(stat04_3)
print(crit_vals04_3)
print(sig_level04_3)
print('----')

print('model 4')
print(stat04_4)
print(crit_vals04_4)
print(sig_level04_4)
print('----')

print('model 5')
print(stat04_5)
print(crit_vals04_5)
print(sig_level04_5)
print('----')

print('model 6')
print(stat04_6)
print(crit_vals04_6)
print(sig_level04_6)
print('----')

print('model 7')
print(stat04_7)
print(crit_vals04_7)
print(sig_level04_7)
print('----')

print('model 8')
print(stat04_8)
print(crit_vals04_8)
print(sig_level04_8)





0.2 < 0.25
----
model 1
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 2
205.09409552239845
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 3
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 4
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 5
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 6
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 7
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 8
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----

----
----

0.25 < 0.3
----
model 1
232.37891425792532
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 2
150.82017887598425
[0.325 1.226 1.961 2.718 3.752]
1.9642124941063038e+181
----
model 3
232.37891425792532
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 4
232.37891425792532
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 5
232.37891425792532
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 6
232.37891425792532
[0.325 1.226 1.961 2.718 

In [45]:
# perform k-sample K-S test on model distributions and data

from scipy import stats

stat25_1, pval25_1 = stats.ks_2samp(sumsat25_model1, sumsat25_data)
stat25_2, pval25_2 = stats.ks_2samp(sumsat25_model2, sumsat25_data)
stat25_3, pval25_3 = stats.ks_2samp(sumsat25_model3, sumsat25_data)
stat25_4, pval25_4 = stats.ks_2samp(sumsat25_model4, sumsat25_data)
stat25_5, pval25_5 = stats.ks_2samp(sumsat25_model5, sumsat25_data)
stat25_6, pval25_6 = stats.ks_2samp(sumsat25_model6, sumsat25_data)
stat25_7, pval25_7 = stats.ks_2samp(sumsat25_model7, sumsat25_data)
stat25_8, pval25_8 = stats.ks_2samp(sumsat25_model8, sumsat25_data)

stat03_1, pval03_1 = stats.ks_2samp(sumsat03_model1, sumsat03_data)
stat03_2, pval03_2 = stats.ks_2samp(sumsat03_model2, sumsat03_data)
stat03_3, pval03_3 = stats.ks_2samp(sumsat03_model3, sumsat03_data)
stat03_4, pval03_4 = stats.ks_2samp(sumsat03_model4, sumsat03_data)
stat03_5, pval03_5 = stats.ks_2samp(sumsat03_model5, sumsat03_data)
stat03_6, pval03_6 = stats.ks_2samp(sumsat03_model6, sumsat03_data)
stat03_7, pval03_7 = stats.ks_2samp(sumsat03_model7, sumsat03_data)
stat03_8, pval03_8 = stats.ks_2samp(sumsat03_model8, sumsat03_data)

stat35_1, pval35_1 = stats.ks_2samp(sumsat35_model1, sumsat35_data)
stat35_2, pval35_2 = stats.ks_2samp(sumsat35_model2, sumsat35_data)
stat35_3, pval35_3 = stats.ks_2samp(sumsat35_model3, sumsat35_data)
stat35_4, pval35_4 = stats.ks_2samp(sumsat35_model4, sumsat35_data)
stat35_5, pval35_5 = stats.ks_2samp(sumsat35_model5, sumsat35_data)
stat35_6, pval35_6 = stats.ks_2samp(sumsat35_model6, sumsat35_data)
stat35_7, pval35_7 = stats.ks_2samp(sumsat35_model7, sumsat35_data)
stat35_8, pval35_8 = stats.ks_2samp(sumsat35_model8, sumsat35_data)

stat04_1, pval04_1 = stats.ks_2samp(sumsat04_model1, sumsat04_data)
stat04_2, pval04_2 = stats.ks_2samp(sumsat04_model2, sumsat04_data)
stat04_3, pval04_3 = stats.ks_2samp(sumsat04_model3, sumsat04_data)
stat04_4, pval04_4 = stats.ks_2samp(sumsat04_model4, sumsat04_data)
stat04_5, pval04_5 = stats.ks_2samp(sumsat04_model5, sumsat04_data)
stat04_6, pval04_6 = stats.ks_2samp(sumsat04_model6, sumsat04_data)
stat04_7, pval04_7 = stats.ks_2samp(sumsat04_model7, sumsat04_data)
stat04_8, pval04_8 = stats.ks_2samp(sumsat04_model8, sumsat04_data)

print('0.2 < 0.25')
print('----')
print('model 1')
print(stat25_1)
print(pval25_1)
print('----')

print('model 2')
print(stat25_2)
print(pval25_2)
print('----')

print('model 3')
print(stat25_3)
print(pval25_3)
print('----')

print('model 4')
print(stat25_4)
print(pval25_4)
print('----')

print('model 5')
print(stat25_5)
print(pval25_5)
print('----')

print('model 6')
print(stat25_6)
print(pval25_6)
print('----')

print('model 7')
print(stat25_7)
print(pval25_7)
print('----')

print('model 8')
print(stat25_8)
print(pval25_8)
print('----')
print()
print('----')
print('----')
print()
print('0.25 < 0.3')
print('----')
print('model 1')
print(stat03_1)
print(pval03_1)
print('----')

print('model 2')
print(stat03_2)
print(pval03_2)
print('----')

print('model 3')
print(stat03_3)
print(pval03_3)
print('----')

print('model 4')
print(stat03_4)
print(pval03_4)
print('----')

print('model 5')
print(stat03_5)
print(pval03_5)
print('----')

print('model 6')
print(stat03_6)
print(pval03_6)
print('----')

print('model 7')
print(stat03_7)
print(pval03_7)
print('----')

print('model 8')
print(stat03_8)
print(pval03_8)
print('----')
print()
print('----')
print('----')
print()
print('0.3 < 0.35')
print('----')
print('model 1')
print(stat35_1)
print(pval35_1)
print('----')

print('model 2')
print(stat35_2)
print(pval35_2)
print('----')

print('model 3')
print(stat35_3)
print(pval35_3)
print('----')

print('model 4')
print(stat35_4)
print(pval35_4)
print('----')

print('model 5')
print(stat35_5)
print(pval35_5)
print('----')

print('model 6')
print(stat35_6)
print(pval35_6)
print('----')

print('model 7')
print(stat35_7)
print(pval35_7)
print('----')

print('model 8')
print(stat35_8)
print(pval35_8)
print('----')
print()
print('----')
print('----')
print()
print('0.35 < 0.4')
print('----')
print('model 1')
print(stat04_1)
print(pval04_1)
print('----')

print('model 2')
print(stat04_2)
print(pval04_2)
print('----')

print('model 3')
print(stat04_3)
print(pval04_3)
print('----')

print('model 4')
print(stat04_4)
print(pval04_4)
print('----')

print('model 5')
print(stat04_5)
print(pval04_5)
print('----')

print('model 6')
print(stat04_6)
print(pval04_6)
print('----')

print('model 7')
print(stat04_7)
print(pval04_7)
print('----')

print('model 8')
print(stat04_8)
print(pval04_8)





0.2 < 0.25
----
model 1
0.8769891342446547
3.417113128532092e-58
----
model 2
0.5016473887136347
2.5104228889830655e-19
----
model 3
0.9555555555555556
5.244153427209231e-69
----
model 4
0.9555555555555556
5.244153427209231e-69
----
model 5
0.9888888888888889
7.103540366509373e-74
----
model 6
0.9888888888888889
7.103540366509373e-74
----
model 7
1.0
1.5547132105820627e-75
----
model 8
1.0
1.5547132105820627e-75
----

----
----

0.25 < 0.3
----
model 1
0.901242916329349
4.8747933621224124e-57
----
model 2
0.43471593885423115
1.346666683501369e-13
----
model 3
0.9879518072289156
1.8637800015164047e-68
----
model 4
0.9879518072289156
1.8637800015164047e-68
----
model 5
1.0
3.990187229608255e-70
----
model 6
1.0
3.990187229608255e-70
----
model 7
1.0
3.990187229608255e-70
----
model 8
1.0
3.990187229608255e-70
----

----
----

0.3 < 0.35
----
model 1
0.6556545153272577
3.82822356289434e-47
----
model 2
0.26833057166528584
2.993663490514721e-08
----
model 3
0.7967067108533554
2.08799035189

In [46]:
# model comparision for mid2

# models for 0.2 < 0.25

sumsat25_model1 = totalNsat(mid2_model25_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat25_model2 = totalNsat(mid2_model25_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat25_model3 = totalNsat(mid2_model25_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat25_model4 = totalNsat(mid2_model25_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat25_model5 = totalNsat(mid2_model25_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat25_model6 = totalNsat(mid2_model25_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat25_model7 = totalNsat(mid2_model25_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat25_model8 = totalNsat(mid2_model25_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.25 < 0.3

sumsat03_model1 = totalNsat(mid2_model03_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat03_model2 = totalNsat(mid2_model03_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat03_model3 = totalNsat(mid2_model03_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat03_model4 = totalNsat(mid2_model03_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat03_model5 = totalNsat(mid2_model03_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat03_model6 = totalNsat(mid2_model03_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat03_model7 = totalNsat(mid2_model03_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat03_model8 = totalNsat(mid2_model03_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.3 < 0.35

sumsat35_model1 = totalNsat(mid2_model35_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat35_model2 = totalNsat(mid2_model35_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat35_model3 = totalNsat(mid2_model35_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat35_model4 = totalNsat(mid2_model35_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat35_model5 = totalNsat(mid2_model35_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat35_model6 = totalNsat(mid2_model35_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat35_model7 = totalNsat(mid2_model35_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat35_model8 = totalNsat(mid2_model35_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.35 < 0.4

sumsat04_model1 = totalNsat(mid2_model04_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat04_model2 = totalNsat(mid2_model04_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat04_model3 = totalNsat(mid2_model04_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat04_model4 = totalNsat(mid2_model04_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat04_model5 = totalNsat(mid2_model04_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat04_model6 = totalNsat(mid2_model04_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat04_model7 = totalNsat(mid2_model04_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat04_model8 = totalNsat(mid2_model04_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.4 < 0.45

sumsat45_model1 = totalNsat(mid2_model45_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat45_model2 = totalNsat(mid2_model45_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat45_model3 = totalNsat(mid2_model45_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat45_model4 = totalNsat(mid2_model45_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat45_model5 = totalNsat(mid2_model45_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat45_model6 = totalNsat(mid2_model45_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat45_model7 = totalNsat(mid2_model45_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat45_model8 = totalNsat(mid2_model45_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.45 < 0.5

sumsat05_model1 = totalNsat(mid2_model05_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat05_model2 = totalNsat(mid2_model05_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat05_model3 = totalNsat(mid2_model05_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat05_model4 = totalNsat(mid2_model05_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat05_model5 = totalNsat(mid2_model05_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat05_model6 = totalNsat(mid2_model05_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat05_model7 = totalNsat(mid2_model05_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat05_model8 = totalNsat(mid2_model05_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')


# data

sumsat25_data = totalNsat(mid2_maskedNsat25, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat03_data = totalNsat(mid2_maskedNsat03, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat35_data = totalNsat(mid2_maskedNsat35, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat04_data = totalNsat(mid2_maskedNsat04, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat45_data = totalNsat(mid2_maskedNsat45, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat05_data = totalNsat(mid2_maskedNsat05, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')


mean number of satellites is 3.00684694583494
mean number of satellites is 1.0068469458349403
mean number of satellites is 4.0068469458349405
mean number of satellites is 4.0068469458349405
mean number of satellites is 5.00684694583494
mean number of satellites is 5.00684694583494
mean number of satellites is 7.00684694583494
mean number of satellites is 7.00684694583494
----
mean number of satellites is 3.051138044422949
mean number of satellites is 1.0511380444229486
mean number of satellites is 4.051138044422949
mean number of satellites is 4.051138044422949
mean number of satellites is 5.051138044422949
mean number of satellites is 5.051138044422949
mean number of satellites is 7.051138044422949
mean number of satellites is 7.051138044422949
----
mean number of satellites is 3.033115606910733
mean number of satellites is 1.0331156069107326
mean number of satellites is 4.033115606910733
mean number of satellites is 4.033115606910733
mean number of satellites is 5.033115606910733
mea

In [35]:
# perform k-sample Anderson-Darling test on model distributions and data

from scipy import stats

stat25_1, crit_vals25_1, sig_level25_1 = stats.anderson_ksamp([sumsat25_model1, sumsat25_data])
stat25_2, crit_vals25_2, sig_level25_2 = stats.anderson_ksamp([sumsat25_model2, sumsat25_data])
stat25_3, crit_vals25_3, sig_level25_3 = stats.anderson_ksamp([sumsat25_model3, sumsat25_data])
stat25_4, crit_vals25_4, sig_level25_4 = stats.anderson_ksamp([sumsat25_model4, sumsat25_data])
stat25_5, crit_vals25_5, sig_level25_5 = stats.anderson_ksamp([sumsat25_model5, sumsat25_data])
stat25_6, crit_vals25_6, sig_level25_6 = stats.anderson_ksamp([sumsat25_model6, sumsat25_data])
stat25_7, crit_vals25_7, sig_level25_7 = stats.anderson_ksamp([sumsat25_model7, sumsat25_data])
stat25_8, crit_vals25_8, sig_level25_8 = stats.anderson_ksamp([sumsat25_model8, sumsat25_data])

stat03_1, crit_vals03_1, sig_level03_1 = stats.anderson_ksamp([sumsat03_model1, sumsat03_data])
stat03_2, crit_vals03_2, sig_level03_2 = stats.anderson_ksamp([sumsat03_model2, sumsat03_data])
stat03_3, crit_vals03_3, sig_level03_3 = stats.anderson_ksamp([sumsat03_model3, sumsat03_data])
stat03_4, crit_vals03_4, sig_level03_4 = stats.anderson_ksamp([sumsat03_model4, sumsat03_data])
stat03_5, crit_vals03_5, sig_level03_5 = stats.anderson_ksamp([sumsat03_model5, sumsat03_data])
stat03_6, crit_vals03_6, sig_level03_6 = stats.anderson_ksamp([sumsat03_model6, sumsat03_data])
stat03_7, crit_vals03_7, sig_level03_7 = stats.anderson_ksamp([sumsat03_model7, sumsat03_data])
stat03_8, crit_vals03_8, sig_level03_8 = stats.anderson_ksamp([sumsat03_model8, sumsat03_data])

stat35_1, crit_vals35_1, sig_level35_1 = stats.anderson_ksamp([sumsat35_model1, sumsat35_data])
stat35_2, crit_vals35_2, sig_level35_2 = stats.anderson_ksamp([sumsat35_model2, sumsat35_data])
stat35_3, crit_vals35_3, sig_level35_3 = stats.anderson_ksamp([sumsat35_model3, sumsat35_data])
stat35_4, crit_vals35_4, sig_level35_4 = stats.anderson_ksamp([sumsat35_model4, sumsat35_data])
stat35_5, crit_vals35_5, sig_level35_5 = stats.anderson_ksamp([sumsat35_model5, sumsat35_data])
stat35_6, crit_vals35_6, sig_level35_6 = stats.anderson_ksamp([sumsat35_model6, sumsat35_data])
stat35_7, crit_vals35_7, sig_level35_7 = stats.anderson_ksamp([sumsat35_model7, sumsat35_data])
stat35_8, crit_vals35_8, sig_level35_8 = stats.anderson_ksamp([sumsat35_model8, sumsat35_data])

stat04_1, crit_vals04_1, sig_level04_1 = stats.anderson_ksamp([sumsat04_model1, sumsat04_data])
stat04_2, crit_vals04_2, sig_level04_2 = stats.anderson_ksamp([sumsat04_model2, sumsat04_data])
stat04_3, crit_vals04_3, sig_level04_3 = stats.anderson_ksamp([sumsat04_model3, sumsat04_data])
stat04_4, crit_vals04_4, sig_level04_4 = stats.anderson_ksamp([sumsat04_model4, sumsat04_data])
stat04_5, crit_vals04_5, sig_level04_5 = stats.anderson_ksamp([sumsat04_model5, sumsat04_data])
stat04_6, crit_vals04_6, sig_level04_6 = stats.anderson_ksamp([sumsat04_model6, sumsat04_data])
stat04_7, crit_vals04_7, sig_level04_7 = stats.anderson_ksamp([sumsat04_model7, sumsat04_data])
stat04_8, crit_vals04_8, sig_level04_8 = stats.anderson_ksamp([sumsat04_model8, sumsat04_data])

stat45_1, crit_vals45_1, sig_level45_1 = stats.anderson_ksamp([sumsat45_model1, sumsat45_data])
stat45_2, crit_vals45_2, sig_level45_2 = stats.anderson_ksamp([sumsat45_model2, sumsat45_data])
stat45_3, crit_vals45_3, sig_level45_3 = stats.anderson_ksamp([sumsat45_model3, sumsat45_data])
stat45_4, crit_vals45_4, sig_level45_4 = stats.anderson_ksamp([sumsat45_model4, sumsat45_data])
stat45_5, crit_vals45_5, sig_level45_5 = stats.anderson_ksamp([sumsat45_model5, sumsat45_data])
stat45_6, crit_vals45_6, sig_level45_6 = stats.anderson_ksamp([sumsat45_model6, sumsat45_data])
stat45_7, crit_vals45_7, sig_level45_7 = stats.anderson_ksamp([sumsat45_model7, sumsat45_data])
stat45_8, crit_vals45_8, sig_level45_8 = stats.anderson_ksamp([sumsat45_model8, sumsat45_data])

stat05_1, crit_vals05_1, sig_level05_1 = stats.anderson_ksamp([sumsat05_model1, sumsat05_data])
stat05_2, crit_vals05_2, sig_level05_2 = stats.anderson_ksamp([sumsat05_model2, sumsat05_data])
stat05_3, crit_vals05_3, sig_level05_3 = stats.anderson_ksamp([sumsat05_model3, sumsat05_data])
stat05_4, crit_vals05_4, sig_level05_4 = stats.anderson_ksamp([sumsat05_model4, sumsat05_data])
stat05_5, crit_vals05_5, sig_level05_5 = stats.anderson_ksamp([sumsat05_model5, sumsat05_data])
stat05_6, crit_vals05_6, sig_level05_6 = stats.anderson_ksamp([sumsat05_model6, sumsat05_data])
stat05_7, crit_vals05_7, sig_level05_7 = stats.anderson_ksamp([sumsat05_model7, sumsat05_data])
stat05_8, crit_vals05_8, sig_level05_8 = stats.anderson_ksamp([sumsat05_model8, sumsat05_data])

print('0.2 < 0.25')
print('----')
print('model 1')
print(stat25_1)
print(crit_vals25_1)
print(sig_level25_1)
print('----')

print('model 2')
print(stat25_2)
print(crit_vals25_2)
print(sig_level25_2)
print('----')

print('model 3')
print(stat25_3)
print(crit_vals25_3)
print(sig_level25_3)
print('----')

print('model 4')
print(stat25_4)
print(crit_vals25_4)
print(sig_level25_4)
print('----')

print('model 5')
print(stat25_5)
print(crit_vals25_5)
print(sig_level25_5)
print('----')

print('model 6')
print(stat25_6)
print(crit_vals25_6)
print(sig_level25_6)
print('----')

print('model 7')
print(stat25_7)
print(crit_vals25_7)
print(sig_level25_7)
print('----')

print('model 8')
print(stat25_8)
print(crit_vals25_8)
print(sig_level25_8)
print('----')
print()
print('----')
print('----')
print()
print('0.25 < 0.3')
print('----')
print('model 1')
print(stat03_1)
print(crit_vals03_1)
print(sig_level03_1)
print('----')

print('model 2')
print(stat03_2)
print(crit_vals03_2)
print(sig_level03_2)
print('----')

print('model 3')
print(stat03_3)
print(crit_vals03_3)
print(sig_level03_3)
print('----')

print('model 4')
print(stat03_4)
print(crit_vals03_4)
print(sig_level03_4)
print('----')

print('model 5')
print(stat03_5)
print(crit_vals03_5)
print(sig_level03_5)
print('----')

print('model 6')
print(stat03_6)
print(crit_vals03_6)
print(sig_level03_6)
print('----')

print('model 7')
print(stat03_7)
print(crit_vals03_7)
print(sig_level03_7)
print('----')

print('model 8')
print(stat03_8)
print(crit_vals03_8)
print(sig_level03_8)
print('----')
print()
print('----')
print('----')
print()
print('0.3 < 0.35')
print('----')
print('model 1')
print(stat35_1)
print(crit_vals35_1)
print(sig_level35_1)
print('----')

print('model 2')
print(stat35_2)
print(crit_vals35_2)
print(sig_level35_2)
print('----')

print('model 3')
print(stat35_3)
print(crit_vals35_3)
print(sig_level35_3)
print('----')

print('model 4')
print(stat35_4)
print(crit_vals35_4)
print(sig_level35_4)
print('----')

print('model 5')
print(stat35_5)
print(crit_vals35_5)
print(sig_level35_5)
print('----')

print('model 6')
print(stat35_6)
print(crit_vals35_6)
print(sig_level35_6)
print('----')

print('model 7')
print(stat35_7)
print(crit_vals35_7)
print(sig_level35_7)
print('----')

print('model 8')
print(stat35_8)
print(crit_vals35_8)
print(sig_level35_8)
print('----')
print()
print('----')
print('----')
print()
print('0.35 < 0.4')
print('----')
print('model 1')
print(stat04_1)
print(crit_vals04_1)
print(sig_level04_1)
print('----')

print('model 2')
print(stat04_2)
print(crit_vals04_2)
print(sig_level04_2)
print('----')

print('model 3')
print(stat04_3)
print(crit_vals04_3)
print(sig_level04_3)
print('----')

print('model 4')
print(stat04_4)
print(crit_vals04_4)
print(sig_level04_4)
print('----')

print('model 5')
print(stat04_5)
print(crit_vals04_5)
print(sig_level04_5)
print('----')

print('model 6')
print(stat04_6)
print(crit_vals04_6)
print(sig_level04_6)
print('----')

print('model 7')
print(stat04_7)
print(crit_vals04_7)
print(sig_level04_7)
print('----')

print('model 8')
print(stat04_8)
print(crit_vals04_8)
print(sig_level04_8)
print('----')
print()
print('----')
print('----')
print()
print('0.4 < 0.45')
print('----')
print('model 1')
print(stat45_1)
print(crit_vals45_1)
print(sig_level45_1)
print('----')

print('model 2')
print(stat45_2)
print(crit_vals45_2)
print(sig_level45_2)
print('----')

print('model 3')
print(stat45_3)
print(crit_vals45_3)
print(sig_level45_3)
print('----')

print('model 4')
print(stat45_4)
print(crit_vals45_4)
print(sig_level45_4)
print('----')

print('model 5')
print(stat45_5)
print(crit_vals45_5)
print(sig_level45_5)
print('----')

print('model 6')
print(stat45_6)
print(crit_vals45_6)
print(sig_level45_6)
print('----')

print('model 7')
print(stat45_7)
print(crit_vals45_7)
print(sig_level45_7)
print('----')

print('model 8')
print(stat45_8)
print(crit_vals45_8)
print(sig_level45_8)
print('----')
print()
print('----')
print('----')
print()
print('0.45 < 0.5')
print('----')
print('model 1')
print(stat05_1)
print(crit_vals05_1)
print(sig_level05_1)
print('----')

print('model 2')
print(stat05_2)
print(crit_vals05_2)
print(sig_level05_2)
print('----')

print('model 3')
print(stat05_3)
print(crit_vals05_3)
print(sig_level05_3)
print('----')

print('model 4')
print(stat05_4)
print(crit_vals05_4)
print(sig_level05_4)
print('----')

print('model 5')
print(stat05_5)
print(crit_vals05_5)
print(sig_level05_5)
print('----')

print('model 6')
print(stat05_6)
print(crit_vals05_6)
print(sig_level05_6)
print('----')

print('model 7')
print(stat05_7)
print(crit_vals05_7)
print(sig_level05_7)
print('----')

print('model 8')
print(stat05_8)
print(crit_vals05_8)
print(sig_level05_8)



0.2 < 0.25
----
model 1
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 2
205.09409552239845
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 3
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 4
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 5
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 6
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 7
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 8
247.20043589751924
[0.325 1.226 1.961 2.718 3.752]
inf
----

----
----

0.25 < 0.3
----
model 1
232.37891425792532
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 2
150.82017887598425
[0.325 1.226 1.961 2.718 3.752]
1.9642124941063038e+181
----
model 3
232.37891425792532
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 4
232.37891425792532
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 5
232.37891425792532
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 6
232.37891425792532
[0.325 1.226 1.961 2.718 

In [47]:
# perform k-sample K-S test on model distributions and data

from scipy import stats

stat25_1, pval25_1 = stats.ks_2samp(sumsat25_model1, sumsat25_data)
stat25_2, pval25_2 = stats.ks_2samp(sumsat25_model2, sumsat25_data)
stat25_3, pval25_3 = stats.ks_2samp(sumsat25_model3, sumsat25_data)
stat25_4, pval25_4 = stats.ks_2samp(sumsat25_model4, sumsat25_data)
stat25_5, pval25_5 = stats.ks_2samp(sumsat25_model5, sumsat25_data)
stat25_6, pval25_6 = stats.ks_2samp(sumsat25_model6, sumsat25_data)
stat25_7, pval25_7 = stats.ks_2samp(sumsat25_model7, sumsat25_data)
stat25_8, pval25_8 = stats.ks_2samp(sumsat25_model8, sumsat25_data)

stat03_1, pval03_1 = stats.ks_2samp(sumsat03_model1, sumsat03_data)
stat03_2, pval03_2 = stats.ks_2samp(sumsat03_model2, sumsat03_data)
stat03_3, pval03_3 = stats.ks_2samp(sumsat03_model3, sumsat03_data)
stat03_4, pval03_4 = stats.ks_2samp(sumsat03_model4, sumsat03_data)
stat03_5, pval03_5 = stats.ks_2samp(sumsat03_model5, sumsat03_data)
stat03_6, pval03_6 = stats.ks_2samp(sumsat03_model6, sumsat03_data)
stat03_7, pval03_7 = stats.ks_2samp(sumsat03_model7, sumsat03_data)
stat03_8, pval03_8 = stats.ks_2samp(sumsat03_model8, sumsat03_data)

stat35_1, pval35_1 = stats.ks_2samp(sumsat35_model1, sumsat35_data)
stat35_2, pval35_2 = stats.ks_2samp(sumsat35_model2, sumsat35_data)
stat35_3, pval35_3 = stats.ks_2samp(sumsat35_model3, sumsat35_data)
stat35_4, pval35_4 = stats.ks_2samp(sumsat35_model4, sumsat35_data)
stat35_5, pval35_5 = stats.ks_2samp(sumsat35_model5, sumsat35_data)
stat35_6, pval35_6 = stats.ks_2samp(sumsat35_model6, sumsat35_data)
stat35_7, pval35_7 = stats.ks_2samp(sumsat35_model7, sumsat35_data)
stat35_8, pval35_8 = stats.ks_2samp(sumsat35_model8, sumsat35_data)

stat04_1, pval04_1 = stats.ks_2samp(sumsat04_model1, sumsat04_data)
stat04_2, pval04_2 = stats.ks_2samp(sumsat04_model2, sumsat04_data)
stat04_3, pval04_3 = stats.ks_2samp(sumsat04_model3, sumsat04_data)
stat04_4, pval04_4 = stats.ks_2samp(sumsat04_model4, sumsat04_data)
stat04_5, pval04_5 = stats.ks_2samp(sumsat04_model5, sumsat04_data)
stat04_6, pval04_6 = stats.ks_2samp(sumsat04_model6, sumsat04_data)
stat04_7, pval04_7 = stats.ks_2samp(sumsat04_model7, sumsat04_data)
stat04_8, pval04_8 = stats.ks_2samp(sumsat04_model8, sumsat04_data)

stat45_1, pval45_1 = stats.ks_2samp(sumsat45_model1, sumsat45_data)
stat45_2, pval45_2 = stats.ks_2samp(sumsat45_model2, sumsat45_data)
stat45_3, pval45_3 = stats.ks_2samp(sumsat45_model3, sumsat45_data)
stat45_4, pval45_4 = stats.ks_2samp(sumsat45_model4, sumsat45_data)
stat45_5, pval45_5 = stats.ks_2samp(sumsat45_model5, sumsat45_data)
stat45_6, pval45_6 = stats.ks_2samp(sumsat45_model6, sumsat45_data)
stat45_7, pval45_7 = stats.ks_2samp(sumsat45_model7, sumsat45_data)
stat45_8, pval45_8 = stats.ks_2samp(sumsat45_model8, sumsat45_data)

stat05_1, pval05_1 = stats.ks_2samp(sumsat05_model1, sumsat05_data)
stat05_2, pval05_2 = stats.ks_2samp(sumsat05_model2, sumsat05_data)
stat05_3, pval05_3 = stats.ks_2samp(sumsat05_model3, sumsat05_data)
stat05_4, pval05_4 = stats.ks_2samp(sumsat05_model4, sumsat05_data)
stat05_5, pval05_5 = stats.ks_2samp(sumsat05_model5, sumsat05_data)
stat05_6, pval05_6 = stats.ks_2samp(sumsat05_model6, sumsat05_data)
stat05_7, pval05_7 = stats.ks_2samp(sumsat05_model7, sumsat05_data)
stat05_8, pval05_8 = stats.ks_2samp(sumsat05_model8, sumsat05_data)

print('0.2 < 0.25')
print('----')
print('model 1')
print(stat25_1)
print(pval25_1)
print('----')

print('model 2')
print(stat25_2)
print(pval25_2)
print('----')

print('model 3')
print(stat25_3)
print(pval25_3)
print('----')

print('model 4')
print(stat25_4)
print(pval25_4)
print('----')

print('model 5')
print(stat25_5)
print(pval25_5)
print('----')

print('model 6')
print(stat25_6)
print(pval25_6)
print('----')

print('model 7')
print(stat25_7)
print(pval25_7)
print('----')

print('model 8')
print(stat25_8)
print(pval25_8)
print('----')
print()
print('----')
print('----')
print()
print('0.25 < 0.3')
print('----')
print('model 1')
print(stat03_1)
print(pval03_1)
print('----')

print('model 2')
print(stat03_2)
print(pval03_2)
print('----')

print('model 3')
print(stat03_3)
print(pval03_3)
print('----')

print('model 4')
print(stat03_4)
print(pval03_4)
print('----')

print('model 5')
print(stat03_5)
print(pval03_5)
print('----')

print('model 6')
print(stat03_6)
print(pval03_6)
print('----')

print('model 7')
print(stat03_7)
print(pval03_7)
print('----')

print('model 8')
print(stat03_8)
print(pval03_8)
print('----')
print()
print('----')
print('----')
print()
print('0.3 < 0.35')
print('----')
print('model 1')
print(stat35_1)
print(pval35_1)
print('----')

print('model 2')
print(stat35_2)
print(pval35_2)
print('----')

print('model 3')
print(stat35_3)
print(pval35_3)
print('----')

print('model 4')
print(stat35_4)
print(pval35_4)
print('----')

print('model 5')
print(stat35_5)
print(pval35_5)
print('----')

print('model 6')
print(stat35_6)
print(pval35_6)
print('----')

print('model 7')
print(stat35_7)
print(pval35_7)
print('----')

print('model 8')
print(stat35_8)
print(pval35_8)
print('----')
print()
print('----')
print('----')
print()
print('0.35 < 0.4')
print('----')
print('model 1')
print(stat04_1)
print(pval04_1)
print('----')

print('model 2')
print(stat04_2)
print(pval04_2)
print('----')

print('model 3')
print(stat04_3)
print(pval04_3)
print('----')

print('model 4')
print(stat04_4)
print(pval04_4)
print('----')

print('model 5')
print(stat04_5)
print(pval04_5)
print('----')

print('model 6')
print(stat04_6)
print(pval04_6)
print('----')

print('model 7')
print(stat04_7)
print(pval04_7)
print('----')

print('model 8')
print(stat04_8)
print(pval04_8)
print('----')
print()
print('----')
print('----')
print()
print('0.4 < 0.45')
print('----')
print('model 1')
print(stat45_1)
print(pval45_1)
print('----')

print('model 2')
print(stat45_2)
print(pval45_2)
print('----')

print('model 3')
print(stat45_3)
print(pval45_3)
print('----')

print('model 4')
print(stat45_4)
print(pval45_4)
print('----')

print('model 5')
print(stat45_5)
print(pval45_5)
print('----')

print('model 6')
print(stat45_6)
print(pval45_6)
print('----')

print('model 7')
print(stat45_7)
print(pval45_7)
print('----')

print('model 8')
print(stat45_8)
print(pval45_8)
print('----')
print()
print('----')
print('----')
print()
print('0.45 < 0.5')
print('----')
print('model 1')
print(stat05_1)
print(pval05_1)
print('----')

print('model 2')
print(stat05_2)
print(pval05_2)
print('----')

print('model 3')
print(stat05_3)
print(pval05_3)
print('----')

print('model 4')
print(stat05_4)
print(pval05_4)
print('----')

print('model 5')
print(stat05_5)
print(pval05_5)
print('----')

print('model 6')
print(stat05_6)
print(pval05_6)
print('----')

print('model 7')
print(stat05_7)
print(pval05_7)
print('----')

print('model 8')
print(stat05_8)
print(pval05_8)




0.2 < 0.25
----
model 1
1.0
2.2913679813616312e-73
----
model 2
0.8888888888888888
4.663222836072999e-58
----
model 3
1.0
2.2913679813616312e-73
----
model 4
1.0
2.2913679813616312e-73
----
model 5
1.0
2.2913679813616312e-73
----
model 6
1.0
2.2913679813616312e-73
----
model 7
1.0
2.2913679813616312e-73
----
model 8
1.0
2.2913679813616312e-73
----

----
----

0.25 < 0.3
----
model 1
1.0
4.2016844928207564e-68
----
model 2
0.7590361445783133
2.0395452482022985e-39
----
model 3
1.0
4.2016844928207564e-68
----
model 4
1.0
4.2016844928207564e-68
----
model 5
1.0
4.2016844928207564e-68
----
model 6
1.0
4.2016844928207564e-68
----
model 7
1.0
4.2016844928207564e-68
----
model 8
1.0
4.2016844928207564e-68
----

----
----

0.3 < 0.35
----
model 1
0.9779411764705882
1.7991798740195662e-99
----
model 2
0.6838235294117647
7.459586453149843e-49
----
model 3
0.9852941176470589
5.755226557671313e-101
----
model 4
0.9852941176470589
5.755226557671313e-101
----
model 5
1.0
5.450575077050226e-104
----


In [48]:
# model comparision for mid3

# models for 0.2 < 0.25

sumsat25_model1 = totalNsat(mid3_model25_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat25_model2 = totalNsat(mid3_model25_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat25_model3 = totalNsat(mid3_model25_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat25_model4 = totalNsat(mid3_model25_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat25_model5 = totalNsat(mid3_model25_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat25_model6 = totalNsat(mid3_model25_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat25_model7 = totalNsat(mid3_model25_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat25_model8 = totalNsat(mid3_model25_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.25 < 0.3

sumsat03_model1 = totalNsat(mid3_model03_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat03_model2 = totalNsat(mid3_model03_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat03_model3 = totalNsat(mid3_model03_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat03_model4 = totalNsat(mid3_model03_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat03_model5 = totalNsat(mid3_model03_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat03_model6 = totalNsat(mid3_model03_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat03_model7 = totalNsat(mid3_model03_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat03_model8 = totalNsat(mid3_model03_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.3 < 0.35

sumsat35_model1 = totalNsat(mid3_model35_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat35_model2 = totalNsat(mid3_model35_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat35_model3 = totalNsat(mid3_model35_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat35_model4 = totalNsat(mid3_model35_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat35_model5 = totalNsat(mid3_model35_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat35_model6 = totalNsat(mid3_model35_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat35_model7 = totalNsat(mid3_model35_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat35_model8 = totalNsat(mid3_model35_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.35 < 0.4

sumsat04_model1 = totalNsat(mid3_model04_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat04_model2 = totalNsat(mid3_model04_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat04_model3 = totalNsat(mid3_model04_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat04_model4 = totalNsat(mid3_model04_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat04_model5 = totalNsat(mid3_model04_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat04_model6 = totalNsat(mid3_model04_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat04_model7 = totalNsat(mid3_model04_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat04_model8 = totalNsat(mid3_model04_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.4 < 0.45

sumsat45_model1 = totalNsat(mid3_model45_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat45_model2 = totalNsat(mid3_model45_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat45_model3 = totalNsat(mid3_model45_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat45_model4 = totalNsat(mid3_model45_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat45_model5 = totalNsat(mid3_model45_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat45_model6 = totalNsat(mid3_model45_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat45_model7 = totalNsat(mid3_model45_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat45_model8 = totalNsat(mid3_model45_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.45 < 0.5

sumsat05_model1 = totalNsat(mid3_model05_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat05_model2 = totalNsat(mid3_model05_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat05_model3 = totalNsat(mid3_model05_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat05_model4 = totalNsat(mid3_model05_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat05_model5 = totalNsat(mid3_model05_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat05_model6 = totalNsat(mid3_model05_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat05_model7 = totalNsat(mid3_model05_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat05_model8 = totalNsat(mid3_model05_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.5 < 0.55

sumsat55_model1 = totalNsat(mid3_model55_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat55_model2 = totalNsat(mid3_model55_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat55_model3 = totalNsat(mid3_model55_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat55_model4 = totalNsat(mid3_model55_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat55_model5 = totalNsat(mid3_model55_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat55_model6 = totalNsat(mid3_model55_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat55_model7 = totalNsat(mid3_model55_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat55_model8 = totalNsat(mid3_model55_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.55 < 0.6

sumsat06_model1 = totalNsat(mid3_model06_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat06_model2 = totalNsat(mid3_model06_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat06_model3 = totalNsat(mid3_model06_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat06_model4 = totalNsat(mid3_model06_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat06_model5 = totalNsat(mid3_model06_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat06_model6 = totalNsat(mid3_model06_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat06_model7 = totalNsat(mid3_model06_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat06_model8 = totalNsat(mid3_model06_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')


# data

sumsat25_data = totalNsat(mid3_maskedNsat25, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat03_data = totalNsat(mid3_maskedNsat03, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat35_data = totalNsat(mid3_maskedNsat35, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat04_data = totalNsat(mid3_maskedNsat04, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat45_data = totalNsat(mid3_maskedNsat45, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat05_data = totalNsat(mid3_maskedNsat05, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat55_data = totalNsat(mid3_maskedNsat55, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat06_data = totalNsat(mid3_maskedNsat06, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')


mean number of satellites is 3.0105563093660725
mean number of satellites is 1.0105563093660723
mean number of satellites is 4.010556309366073
mean number of satellites is 4.010556309366073
mean number of satellites is 5.010556309366073
mean number of satellites is 5.010556309366073
mean number of satellites is 7.010556309366073
mean number of satellites is 7.010556309366073
----
mean number of satellites is 3.0
mean number of satellites is 1.0
mean number of satellites is 4.0
mean number of satellites is 4.0
mean number of satellites is 5.0
mean number of satellites is 5.0
mean number of satellites is 7.0
mean number of satellites is 7.0
----
mean number of satellites is 3.033895630243807
mean number of satellites is 1.0338956302438076
mean number of satellites is 4.033895630243808
mean number of satellites is 4.033895630243808
mean number of satellites is 5.033895630243808
mean number of satellites is 5.033895630243808
mean number of satellites is 7.033895630243808
mean number of sat

In [38]:
# perform k-sample Anderson-Darling test on model distributions and data

from scipy import stats

stat25_1, crit_vals25_1, sig_level25_1 = stats.anderson_ksamp([sumsat25_model1, sumsat25_data])
stat25_2, crit_vals25_2, sig_level25_2 = stats.anderson_ksamp([sumsat25_model2, sumsat25_data])
stat25_3, crit_vals25_3, sig_level25_3 = stats.anderson_ksamp([sumsat25_model3, sumsat25_data])
stat25_4, crit_vals25_4, sig_level25_4 = stats.anderson_ksamp([sumsat25_model4, sumsat25_data])
stat25_5, crit_vals25_5, sig_level25_5 = stats.anderson_ksamp([sumsat25_model5, sumsat25_data])
stat25_6, crit_vals25_6, sig_level25_6 = stats.anderson_ksamp([sumsat25_model6, sumsat25_data])
stat25_7, crit_vals25_7, sig_level25_7 = stats.anderson_ksamp([sumsat25_model7, sumsat25_data])
stat25_8, crit_vals25_8, sig_level25_8 = stats.anderson_ksamp([sumsat25_model8, sumsat25_data])

stat03_1, crit_vals03_1, sig_level03_1 = stats.anderson_ksamp([sumsat03_model1, sumsat03_data])
stat03_2, crit_vals03_2, sig_level03_2 = stats.anderson_ksamp([sumsat03_model2, sumsat03_data])
stat03_3, crit_vals03_3, sig_level03_3 = stats.anderson_ksamp([sumsat03_model3, sumsat03_data])
stat03_4, crit_vals03_4, sig_level03_4 = stats.anderson_ksamp([sumsat03_model4, sumsat03_data])
stat03_5, crit_vals03_5, sig_level03_5 = stats.anderson_ksamp([sumsat03_model5, sumsat03_data])
stat03_6, crit_vals03_6, sig_level03_6 = stats.anderson_ksamp([sumsat03_model6, sumsat03_data])
stat03_7, crit_vals03_7, sig_level03_7 = stats.anderson_ksamp([sumsat03_model7, sumsat03_data])
stat03_8, crit_vals03_8, sig_level03_8 = stats.anderson_ksamp([sumsat03_model8, sumsat03_data])

stat35_1, crit_vals35_1, sig_level35_1 = stats.anderson_ksamp([sumsat35_model1, sumsat35_data])
stat35_2, crit_vals35_2, sig_level35_2 = stats.anderson_ksamp([sumsat35_model2, sumsat35_data])
stat35_3, crit_vals35_3, sig_level35_3 = stats.anderson_ksamp([sumsat35_model3, sumsat35_data])
stat35_4, crit_vals35_4, sig_level35_4 = stats.anderson_ksamp([sumsat35_model4, sumsat35_data])
stat35_5, crit_vals35_5, sig_level35_5 = stats.anderson_ksamp([sumsat35_model5, sumsat35_data])
stat35_6, crit_vals35_6, sig_level35_6 = stats.anderson_ksamp([sumsat35_model6, sumsat35_data])
stat35_7, crit_vals35_7, sig_level35_7 = stats.anderson_ksamp([sumsat35_model7, sumsat35_data])
stat35_8, crit_vals35_8, sig_level35_8 = stats.anderson_ksamp([sumsat35_model8, sumsat35_data])

stat04_1, crit_vals04_1, sig_level04_1 = stats.anderson_ksamp([sumsat04_model1, sumsat04_data])
stat04_2, crit_vals04_2, sig_level04_2 = stats.anderson_ksamp([sumsat04_model2, sumsat04_data])
stat04_3, crit_vals04_3, sig_level04_3 = stats.anderson_ksamp([sumsat04_model3, sumsat04_data])
stat04_4, crit_vals04_4, sig_level04_4 = stats.anderson_ksamp([sumsat04_model4, sumsat04_data])
stat04_5, crit_vals04_5, sig_level04_5 = stats.anderson_ksamp([sumsat04_model5, sumsat04_data])
stat04_6, crit_vals04_6, sig_level04_6 = stats.anderson_ksamp([sumsat04_model6, sumsat04_data])
stat04_7, crit_vals04_7, sig_level04_7 = stats.anderson_ksamp([sumsat04_model7, sumsat04_data])
stat04_8, crit_vals04_8, sig_level04_8 = stats.anderson_ksamp([sumsat04_model8, sumsat04_data])

stat45_1, crit_vals45_1, sig_level45_1 = stats.anderson_ksamp([sumsat45_model1, sumsat45_data])
stat45_2, crit_vals45_2, sig_level45_2 = stats.anderson_ksamp([sumsat45_model2, sumsat45_data])
stat45_3, crit_vals45_3, sig_level45_3 = stats.anderson_ksamp([sumsat45_model3, sumsat45_data])
stat45_4, crit_vals45_4, sig_level45_4 = stats.anderson_ksamp([sumsat45_model4, sumsat45_data])
stat45_5, crit_vals45_5, sig_level45_5 = stats.anderson_ksamp([sumsat45_model5, sumsat45_data])
stat45_6, crit_vals45_6, sig_level45_6 = stats.anderson_ksamp([sumsat45_model6, sumsat45_data])
stat45_7, crit_vals45_7, sig_level45_7 = stats.anderson_ksamp([sumsat45_model7, sumsat45_data])
stat45_8, crit_vals45_8, sig_level45_8 = stats.anderson_ksamp([sumsat45_model8, sumsat45_data])

stat05_1, crit_vals05_1, sig_level05_1 = stats.anderson_ksamp([sumsat05_model1, sumsat05_data])
stat05_2, crit_vals05_2, sig_level05_2 = stats.anderson_ksamp([sumsat05_model2, sumsat05_data])
stat05_3, crit_vals05_3, sig_level05_3 = stats.anderson_ksamp([sumsat05_model3, sumsat05_data])
stat05_4, crit_vals05_4, sig_level05_4 = stats.anderson_ksamp([sumsat05_model4, sumsat05_data])
stat05_5, crit_vals05_5, sig_level05_5 = stats.anderson_ksamp([sumsat05_model5, sumsat05_data])
stat05_6, crit_vals05_6, sig_level05_6 = stats.anderson_ksamp([sumsat05_model6, sumsat05_data])
stat05_7, crit_vals05_7, sig_level05_7 = stats.anderson_ksamp([sumsat05_model7, sumsat05_data])
stat05_8, crit_vals05_8, sig_level05_8 = stats.anderson_ksamp([sumsat05_model8, sumsat05_data])

stat55_1, crit_vals55_1, sig_level55_1 = stats.anderson_ksamp([sumsat55_model1, sumsat55_data])
stat55_2, crit_vals55_2, sig_level55_2 = stats.anderson_ksamp([sumsat55_model2, sumsat55_data])
stat55_3, crit_vals55_3, sig_level55_3 = stats.anderson_ksamp([sumsat55_model3, sumsat55_data])
stat55_4, crit_vals55_4, sig_level55_4 = stats.anderson_ksamp([sumsat55_model4, sumsat55_data])
stat55_5, crit_vals55_5, sig_level55_5 = stats.anderson_ksamp([sumsat55_model5, sumsat55_data])
stat55_6, crit_vals55_6, sig_level55_6 = stats.anderson_ksamp([sumsat55_model6, sumsat55_data])
stat55_7, crit_vals55_7, sig_level55_7 = stats.anderson_ksamp([sumsat55_model7, sumsat55_data])
stat55_8, crit_vals55_8, sig_level55_8 = stats.anderson_ksamp([sumsat55_model8, sumsat55_data])

stat06_1, crit_vals06_1, sig_level06_1 = stats.anderson_ksamp([sumsat06_model1, sumsat06_data])
stat06_2, crit_vals06_2, sig_level06_2 = stats.anderson_ksamp([sumsat06_model2, sumsat06_data])
stat06_3, crit_vals06_3, sig_level06_3 = stats.anderson_ksamp([sumsat06_model3, sumsat06_data])
stat06_4, crit_vals06_4, sig_level06_4 = stats.anderson_ksamp([sumsat06_model4, sumsat06_data])
stat06_5, crit_vals06_5, sig_level06_5 = stats.anderson_ksamp([sumsat06_model5, sumsat06_data])
stat06_6, crit_vals06_6, sig_level06_6 = stats.anderson_ksamp([sumsat06_model6, sumsat06_data])
stat06_7, crit_vals06_7, sig_level06_7 = stats.anderson_ksamp([sumsat06_model7, sumsat06_data])
stat06_8, crit_vals06_8, sig_level06_8 = stats.anderson_ksamp([sumsat06_model8, sumsat06_data])

print('0.2 < 0.25')
print('----')
print('model 1')
print(stat25_1)
print(crit_vals25_1)
print(sig_level25_1)
print('----')

print('model 2')
print(stat25_2)
print(crit_vals25_2)
print(sig_level25_2)
print('----')

print('model 3')
print(stat25_3)
print(crit_vals25_3)
print(sig_level25_3)
print('----')

print('model 4')
print(stat25_4)
print(crit_vals25_4)
print(sig_level25_4)
print('----')

print('model 5')
print(stat25_5)
print(crit_vals25_5)
print(sig_level25_5)
print('----')

print('model 6')
print(stat25_6)
print(crit_vals25_6)
print(sig_level25_6)
print('----')

print('model 7')
print(stat25_7)
print(crit_vals25_7)
print(sig_level25_7)
print('----')

print('model 8')
print(stat25_8)
print(crit_vals25_8)
print(sig_level25_8)
print('----')
print()
print('----')
print('----')
print()
print('0.25 < 0.3')
print('----')
print('model 1')
print(stat03_1)
print(crit_vals03_1)
print(sig_level03_1)
print('----')

print('model 2')
print(stat03_2)
print(crit_vals03_2)
print(sig_level03_2)
print('----')

print('model 3')
print(stat03_3)
print(crit_vals03_3)
print(sig_level03_3)
print('----')

print('model 4')
print(stat03_4)
print(crit_vals03_4)
print(sig_level03_4)
print('----')

print('model 5')
print(stat03_5)
print(crit_vals03_5)
print(sig_level03_5)
print('----')

print('model 6')
print(stat03_6)
print(crit_vals03_6)
print(sig_level03_6)
print('----')

print('model 7')
print(stat03_7)
print(crit_vals03_7)
print(sig_level03_7)
print('----')

print('model 8')
print(stat03_8)
print(crit_vals03_8)
print(sig_level03_8)
print('----')
print()
print('----')
print('----')
print()
print('0.3 < 0.35')
print('----')
print('model 1')
print(stat35_1)
print(crit_vals35_1)
print(sig_level35_1)
print('----')

print('model 2')
print(stat35_2)
print(crit_vals35_2)
print(sig_level35_2)
print('----')

print('model 3')
print(stat35_3)
print(crit_vals35_3)
print(sig_level35_3)
print('----')

print('model 4')
print(stat35_4)
print(crit_vals35_4)
print(sig_level35_4)
print('----')

print('model 5')
print(stat35_5)
print(crit_vals35_5)
print(sig_level35_5)
print('----')

print('model 6')
print(stat35_6)
print(crit_vals35_6)
print(sig_level35_6)
print('----')

print('model 7')
print(stat35_7)
print(crit_vals35_7)
print(sig_level35_7)
print('----')

print('model 8')
print(stat35_8)
print(crit_vals35_8)
print(sig_level35_8)
print('----')
print()
print('----')
print('----')
print()
print('0.35 < 0.4')
print('----')
print('model 1')
print(stat04_1)
print(crit_vals04_1)
print(sig_level04_1)
print('----')

print('model 2')
print(stat04_2)
print(crit_vals04_2)
print(sig_level04_2)
print('----')

print('model 3')
print(stat04_3)
print(crit_vals04_3)
print(sig_level04_3)
print('----')

print('model 4')
print(stat04_4)
print(crit_vals04_4)
print(sig_level04_4)
print('----')

print('model 5')
print(stat04_5)
print(crit_vals04_5)
print(sig_level04_5)
print('----')

print('model 6')
print(stat04_6)
print(crit_vals04_6)
print(sig_level04_6)
print('----')

print('model 7')
print(stat04_7)
print(crit_vals04_7)
print(sig_level04_7)
print('----')

print('model 8')
print(stat04_8)
print(crit_vals04_8)
print(sig_level04_8)
print('----')
print()
print('----')
print('----')
print()
print('0.4 < 0.45')
print('----')
print('model 1')
print(stat45_1)
print(crit_vals45_1)
print(sig_level45_1)
print('----')

print('model 2')
print(stat45_2)
print(crit_vals45_2)
print(sig_level45_2)
print('----')

print('model 3')
print(stat45_3)
print(crit_vals45_3)
print(sig_level45_3)
print('----')

print('model 4')
print(stat45_4)
print(crit_vals45_4)
print(sig_level45_4)
print('----')

print('model 5')
print(stat45_5)
print(crit_vals45_5)
print(sig_level45_5)
print('----')

print('model 6')
print(stat45_6)
print(crit_vals45_6)
print(sig_level45_6)
print('----')

print('model 7')
print(stat45_7)
print(crit_vals45_7)
print(sig_level45_7)
print('----')

print('model 8')
print(stat45_8)
print(crit_vals45_8)
print(sig_level45_8)
print('----')
print()
print('----')
print('----')
print()
print('0.45 < 0.5')
print('----')
print('model 1')
print(stat05_1)
print(crit_vals05_1)
print(sig_level05_1)
print('----')

print('model 2')
print(stat05_2)
print(crit_vals05_2)
print(sig_level05_2)
print('----')

print('model 3')
print(stat05_3)
print(crit_vals05_3)
print(sig_level05_3)
print('----')

print('model 4')
print(stat05_4)
print(crit_vals05_4)
print(sig_level05_4)
print('----')

print('model 5')
print(stat05_5)
print(crit_vals05_5)
print(sig_level05_5)
print('----')

print('model 6')
print(stat05_6)
print(crit_vals05_6)
print(sig_level05_6)
print('----')

print('model 7')
print(stat05_7)
print(crit_vals05_7)
print(sig_level05_7)
print('----')

print('model 8')
print(stat05_8)
print(crit_vals05_8)
print(sig_level05_8)
print('----')
print()
print('----')
print('----')
print()
print('0.5 < 0.55')
print('----')
print('model 1')
print(stat55_1)
print(crit_vals55_1)
print(sig_level55_1)
print('----')

print('model 2')
print(stat55_2)
print(crit_vals55_2)
print(sig_level55_2)
print('----')

print('model 3')
print(stat55_3)
print(crit_vals55_3)
print(sig_level55_3)
print('----')

print('model 4')
print(stat55_4)
print(crit_vals55_4)
print(sig_level55_4)
print('----')

print('model 5')
print(stat55_5)
print(crit_vals55_5)
print(sig_level55_5)
print('----')

print('model 6')
print(stat55_6)
print(crit_vals55_6)
print(sig_level55_6)
print('----')

print('model 7')
print(stat55_7)
print(crit_vals55_7)
print(sig_level55_7)
print('----')

print('model 8')
print(stat55_8)
print(crit_vals55_8)
print(sig_level55_8)
print('----')
print()
print('----')
print('----')
print()
print('0.55 < 0.6')
print('----')
print('model 1')
print(stat06_1)
print(crit_vals06_1)
print(sig_level06_1)
print('----')

print('model 2')
print(stat06_2)
print(crit_vals06_2)
print(sig_level06_2)
print('----')

print('model 3')
print(stat06_3)
print(crit_vals06_3)
print(sig_level06_3)
print('----')

print('model 4')
print(stat06_4)
print(crit_vals06_4)
print(sig_level06_4)
print('----')

print('model 5')
print(stat06_5)
print(crit_vals06_5)
print(sig_level06_5)
print('----')

print('model 6')
print(stat06_6)
print(crit_vals06_6)
print(sig_level06_6)
print('----')

print('model 7')
print(stat06_7)
print(crit_vals06_7)
print(sig_level06_7)
print('----')

print('model 8')
print(stat06_8)
print(crit_vals06_8)
print(sig_level06_8)



0.2 < 0.25
----
model 1
239.18807013665932
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 2
239.18807013665932
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 3
239.18807013665932
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 4
239.18807013665932
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 5
239.18807013665932
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 6
239.18807013665932
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 7
239.18807013665932
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 8
239.18807013665932
[0.325 1.226 1.961 2.718 3.752]
inf
----

----
----

0.25 < 0.3
----
model 1
875.221279715405
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 2
875.221279715405
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 3
875.221279715405
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 4
875.221279715405
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 5
875.221279715405
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 6
875.221279715405
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 7
875.2212

In [49]:
# perform k-sample K-S test on model distributions and data

from scipy import stats

stat25_1, pval25_1 = stats.ks_2samp(sumsat25_model1, sumsat25_data)
stat25_2, pval25_2 = stats.ks_2samp(sumsat25_model2, sumsat25_data)
stat25_3, pval25_3 = stats.ks_2samp(sumsat25_model3, sumsat25_data)
stat25_4, pval25_4 = stats.ks_2samp(sumsat25_model4, sumsat25_data)
stat25_5, pval25_5 = stats.ks_2samp(sumsat25_model5, sumsat25_data)
stat25_6, pval25_6 = stats.ks_2samp(sumsat25_model6, sumsat25_data)
stat25_7, pval25_7 = stats.ks_2samp(sumsat25_model7, sumsat25_data)
stat25_8, pval25_8 = stats.ks_2samp(sumsat25_model8, sumsat25_data)

stat03_1, pval03_1 = stats.ks_2samp(sumsat03_model1, sumsat03_data)
stat03_2, pval03_2 = stats.ks_2samp(sumsat03_model2, sumsat03_data)
stat03_3, pval03_3 = stats.ks_2samp(sumsat03_model3, sumsat03_data)
stat03_4, pval03_4 = stats.ks_2samp(sumsat03_model4, sumsat03_data)
stat03_5, pval03_5 = stats.ks_2samp(sumsat03_model5, sumsat03_data)
stat03_6, pval03_6 = stats.ks_2samp(sumsat03_model6, sumsat03_data)
stat03_7, pval03_7 = stats.ks_2samp(sumsat03_model7, sumsat03_data)
stat03_8, pval03_8 = stats.ks_2samp(sumsat03_model8, sumsat03_data)

stat35_1, pval35_1 = stats.ks_2samp(sumsat35_model1, sumsat35_data)
stat35_2, pval35_2 = stats.ks_2samp(sumsat35_model2, sumsat35_data)
stat35_3, pval35_3 = stats.ks_2samp(sumsat35_model3, sumsat35_data)
stat35_4, pval35_4 = stats.ks_2samp(sumsat35_model4, sumsat35_data)
stat35_5, pval35_5 = stats.ks_2samp(sumsat35_model5, sumsat35_data)
stat35_6, pval35_6 = stats.ks_2samp(sumsat35_model6, sumsat35_data)
stat35_7, pval35_7 = stats.ks_2samp(sumsat35_model7, sumsat35_data)
stat35_8, pval35_8 = stats.ks_2samp(sumsat35_model8, sumsat35_data)

stat04_1, pval04_1 = stats.ks_2samp(sumsat04_model1, sumsat04_data)
stat04_2, pval04_2 = stats.ks_2samp(sumsat04_model2, sumsat04_data)
stat04_3, pval04_3 = stats.ks_2samp(sumsat04_model3, sumsat04_data)
stat04_4, pval04_4 = stats.ks_2samp(sumsat04_model4, sumsat04_data)
stat04_5, pval04_5 = stats.ks_2samp(sumsat04_model5, sumsat04_data)
stat04_6, pval04_6 = stats.ks_2samp(sumsat04_model6, sumsat04_data)
stat04_7, pval04_7 = stats.ks_2samp(sumsat04_model7, sumsat04_data)
stat04_8, pval04_8 = stats.ks_2samp(sumsat04_model8, sumsat04_data)

stat45_1, pval45_1 = stats.ks_2samp(sumsat45_model1, sumsat45_data)
stat45_2, pval45_2 = stats.ks_2samp(sumsat45_model2, sumsat45_data)
stat45_3, pval45_3 = stats.ks_2samp(sumsat45_model3, sumsat45_data)
stat45_4, pval45_4 = stats.ks_2samp(sumsat45_model4, sumsat45_data)
stat45_5, pval45_5 = stats.ks_2samp(sumsat45_model5, sumsat45_data)
stat45_6, pval45_6 = stats.ks_2samp(sumsat45_model6, sumsat45_data)
stat45_7, pval45_7 = stats.ks_2samp(sumsat45_model7, sumsat45_data)
stat45_8, pval45_8 = stats.ks_2samp(sumsat45_model8, sumsat45_data)

stat05_1, pval05_1 = stats.ks_2samp(sumsat05_model1, sumsat05_data)
stat05_2, pval05_2 = stats.ks_2samp(sumsat05_model2, sumsat05_data)
stat05_3, pval05_3 = stats.ks_2samp(sumsat05_model3, sumsat05_data)
stat05_4, pval05_4 = stats.ks_2samp(sumsat05_model4, sumsat05_data)
stat05_5, pval05_5 = stats.ks_2samp(sumsat05_model5, sumsat05_data)
stat05_6, pval05_6 = stats.ks_2samp(sumsat05_model6, sumsat05_data)
stat05_7, pval05_7 = stats.ks_2samp(sumsat05_model7, sumsat05_data)
stat05_8, pval05_8 = stats.ks_2samp(sumsat05_model8, sumsat05_data)

stat55_1, pval55_1 = stats.ks_2samp(sumsat55_model1, sumsat55_data)
stat55_2, pval55_2 = stats.ks_2samp(sumsat55_model2, sumsat55_data)
stat55_3, pval55_3 = stats.ks_2samp(sumsat55_model3, sumsat55_data)
stat55_4, pval55_4 = stats.ks_2samp(sumsat55_model4, sumsat55_data)
stat55_5, pval55_5 = stats.ks_2samp(sumsat55_model5, sumsat55_data)
stat55_6, pval55_6 = stats.ks_2samp(sumsat55_model6, sumsat55_data)
stat55_7, pval55_7 = stats.ks_2samp(sumsat55_model7, sumsat55_data)
stat55_8, pval55_8 = stats.ks_2samp(sumsat55_model8, sumsat55_data)

stat06_1, pval06_1 = stats.ks_2samp(sumsat06_model1, sumsat06_data)
stat06_2, pval06_2 = stats.ks_2samp(sumsat06_model2, sumsat06_data)
stat06_3, pval06_3 = stats.ks_2samp(sumsat06_model3, sumsat06_data)
stat06_4, pval06_4 = stats.ks_2samp(sumsat06_model4, sumsat06_data)
stat06_5, pval06_5 = stats.ks_2samp(sumsat06_model5, sumsat06_data)
stat06_6, pval06_6 = stats.ks_2samp(sumsat06_model6, sumsat06_data)
stat06_7, pval06_7 = stats.ks_2samp(sumsat06_model7, sumsat06_data)
stat06_8, pval06_8 = stats.ks_2samp(sumsat06_model8, sumsat06_data)

print('0.2 < 0.25')
print('----')
print('model 1')
print(stat25_1)
print(pval25_1)
print('----')

print('model 2')
print(stat25_2)
print(pval25_2)
print('----')

print('model 3')
print(stat25_3)
print(pval25_3)
print('----')

print('model 4')
print(stat25_4)
print(pval25_4)
print('----')

print('model 5')
print(stat25_5)
print(pval25_5)
print('----')

print('model 6')
print(stat25_6)
print(pval25_6)
print('----')

print('model 7')
print(stat25_7)
print(pval25_7)
print('----')

print('model 8')
print(stat25_8)
print(pval25_8)
print('----')
print()
print('----')
print('----')
print()
print('0.25 < 0.3')
print('----')
print('model 1')
print(stat03_1)
print(pval03_1)
print('----')

print('model 2')
print(stat03_2)
print(pval03_2)
print('----')

print('model 3')
print(stat03_3)
print(pval03_3)
print('----')

print('model 4')
print(stat03_4)
print(pval03_4)
print('----')

print('model 5')
print(stat03_5)
print(pval03_5)
print('----')

print('model 6')
print(stat03_6)
print(pval03_6)
print('----')

print('model 7')
print(stat03_7)
print(pval03_7)
print('----')

print('model 8')
print(stat03_8)
print(pval03_8)
print('----')
print()
print('----')
print('----')
print()
print('0.3 < 0.35')
print('----')
print('model 1')
print(stat35_1)
print(pval35_1)
print('----')

print('model 2')
print(stat35_2)
print(pval35_2)
print('----')

print('model 3')
print(stat35_3)
print(pval35_3)
print('----')

print('model 4')
print(stat35_4)
print(pval35_4)
print('----')

print('model 5')
print(stat35_5)
print(pval35_5)
print('----')

print('model 6')
print(stat35_6)
print(pval35_6)
print('----')

print('model 7')
print(stat35_7)
print(pval35_7)
print('----')

print('model 8')
print(stat35_8)
print(pval35_8)
print('----')
print()
print('----')
print('----')
print()
print('0.35 < 0.4')
print('----')
print('model 1')
print(stat04_1)
print(pval04_1)
print('----')

print('model 2')
print(stat04_2)
print(pval04_2)
print('----')

print('model 3')
print(stat04_3)
print(pval04_3)
print('----')

print('model 4')
print(stat04_4)
print(pval04_4)
print('----')

print('model 5')
print(stat04_5)
print(pval04_5)
print('----')

print('model 6')
print(stat04_6)
print(pval04_6)
print('----')

print('model 7')
print(stat04_7)
print(pval04_7)
print('----')

print('model 8')
print(stat04_8)
print(pval04_8)
print('----')
print()
print('----')
print('----')
print()
print('0.4 < 0.45')
print('----')
print('model 1')
print(stat45_1)
print(pval45_1)
print('----')

print('model 2')
print(stat45_2)
print(pval45_2)
print('----')

print('model 3')
print(stat45_3)
print(pval45_3)
print('----')

print('model 4')
print(stat45_4)
print(pval45_4)
print('----')

print('model 5')
print(stat45_5)
print(pval45_5)
print('----')

print('model 6')
print(stat45_6)
print(pval45_6)
print('----')

print('model 7')
print(stat45_7)
print(pval45_7)
print('----')

print('model 8')
print(stat45_8)
print(pval45_8)
print('----')
print()
print('----')
print('----')
print()
print('0.45 < 0.5')
print('----')
print('model 1')
print(stat05_1)
print(pval05_1)
print('----')

print('model 2')
print(stat05_2)
print(pval05_2)
print('----')

print('model 3')
print(stat05_3)
print(pval05_3)
print('----')

print('model 4')
print(stat05_4)
print(pval05_4)
print('----')

print('model 5')
print(stat05_5)
print(pval05_5)
print('----')

print('model 6')
print(stat05_6)
print(pval05_6)
print('----')

print('model 7')
print(stat05_7)
print(pval05_7)
print('----')

print('model 8')
print(stat05_8)
print(pval05_8)
print('----')
print()
print('----')
print('----')
print()
print('0.5 < 0.55')
print('----')
print('model 1')
print(stat55_1)
print(pval55_1)
print('----')

print('model 2')
print(stat55_2)
print(pval55_2)
print('----')

print('model 3')
print(stat55_3)
print(pval55_3)
print('----')

print('model 4')
print(stat55_4)
print(pval55_4)
print('----')

print('model 5')
print(stat55_5)
print(pval55_5)
print('----')

print('model 6')
print(stat55_6)
print(pval55_6)
print('----')

print('model 7')
print(stat55_7)
print(pval55_7)
print('----')

print('model 8')
print(stat55_8)
print(pval55_8)
print('----')
print()
print('----')
print('----')
print()
print('0.55 < 0.6')
print('----')
print('model 1')
print(stat06_1)
print(pval06_1)
print('----')

print('model 2')
print(stat06_2)
print(pval06_2)
print('----')

print('model 3')
print(stat06_3)
print(pval06_3)
print('----')

print('model 4')
print(stat06_4)
print(pval06_4)
print('----')

print('model 5')
print(stat06_5)
print(pval06_5)
print('----')

print('model 6')
print(stat06_6)
print(pval06_6)
print('----')

print('model 7')
print(stat06_7)
print(pval06_7)
print('----')

print('model 8')
print(stat06_8)
print(pval06_8)


0.2 < 0.25
----
model 1
1.0
1.0263173711628338e-70
----
model 2
1.0
1.0263173711628338e-70
----
model 3
1.0
1.0263173711628338e-70
----
model 4
1.0
1.0263173711628338e-70
----
model 5
1.0
1.0263173711628338e-70
----
model 6
1.0
1.0263173711628338e-70
----
model 7
1.0
1.0263173711628338e-70
----
model 8
1.0
1.0263173711628338e-70
----

----
----

0.25 < 0.3
----
model 1
1.0
1.569307872105617e-65
----
model 2
1.0
1.569307872105617e-65
----
model 3
1.0
1.569307872105617e-65
----
model 4
1.0
1.569307872105617e-65
----
model 5
1.0
1.569307872105617e-65
----
model 6
1.0
1.569307872105617e-65
----
model 7
1.0
1.569307872105617e-65
----
model 8
1.0
1.569307872105617e-65
----

----
----

0.3 < 0.35
----
model 1
1.0
1.7722398977053665e-100
----
model 2
0.8014705882352942
1.0759477043209319e-64
----
model 3
1.0
1.7722398977053665e-100
----
model 4
1.0
1.7722398977053665e-100
----
model 5
1.0
1.7722398977053665e-100
----
model 6
1.0
1.7722398977053665e-100
----
model 7
1.0
1.7722398977053665e-100


In [50]:
# model comparision for high

# models for 0.2 < 0.25

sumsat25_model1 = totalNsat(high_model25_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat25_model2 = totalNsat(high_model25_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat25_model3 = totalNsat(high_model25_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat25_model4 = totalNsat(high_model25_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat25_model5 = totalNsat(high_model25_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat25_model6 = totalNsat(high_model25_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat25_model7 = totalNsat(high_model25_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat25_model8 = totalNsat(high_model25_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.25 < 0.3

sumsat03_model1 = totalNsat(high_model03_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat03_model2 = totalNsat(high_model03_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat03_model3 = totalNsat(high_model03_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat03_model4 = totalNsat(high_model03_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat03_model5 = totalNsat(high_model03_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat03_model6 = totalNsat(high_model03_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat03_model7 = totalNsat(high_model03_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat03_model8 = totalNsat(high_model03_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.3 < 0.35

sumsat35_model1 = totalNsat(high_model35_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat35_model2 = totalNsat(high_model35_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat35_model3 = totalNsat(high_model35_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat35_model4 = totalNsat(high_model35_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat35_model5 = totalNsat(high_model35_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat35_model6 = totalNsat(high_model35_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat35_model7 = totalNsat(high_model35_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat35_model8 = totalNsat(high_model35_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.35 < 0.4

sumsat04_model1 = totalNsat(high_model04_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat04_model2 = totalNsat(high_model04_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat04_model3 = totalNsat(high_model04_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat04_model4 = totalNsat(high_model04_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat04_model5 = totalNsat(high_model04_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat04_model6 = totalNsat(high_model04_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat04_model7 = totalNsat(high_model04_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat04_model8 = totalNsat(high_model04_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.4 < 0.45

sumsat45_model1 = totalNsat(high_model45_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat45_model2 = totalNsat(high_model45_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat45_model3 = totalNsat(high_model45_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat45_model4 = totalNsat(high_model45_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat45_model5 = totalNsat(high_model45_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat45_model6 = totalNsat(high_model45_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat45_model7 = totalNsat(high_model45_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat45_model8 = totalNsat(high_model45_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.45 < 0.5

sumsat05_model1 = totalNsat(high_model05_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat05_model2 = totalNsat(high_model05_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat05_model3 = totalNsat(high_model05_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat05_model4 = totalNsat(high_model05_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat05_model5 = totalNsat(high_model05_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat05_model6 = totalNsat(high_model05_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat05_model7 = totalNsat(high_model05_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat05_model8 = totalNsat(high_model05_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.5 < 0.55

sumsat55_model1 = totalNsat(high_model55_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat55_model2 = totalNsat(high_model55_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat55_model3 = totalNsat(high_model55_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat55_model4 = totalNsat(high_model55_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat55_model5 = totalNsat(high_model55_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat55_model6 = totalNsat(high_model55_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat55_model7 = totalNsat(high_model55_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat55_model8 = totalNsat(high_model55_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.55 < 0.6

sumsat06_model1 = totalNsat(high_model06_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat06_model2 = totalNsat(high_model06_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat06_model3 = totalNsat(high_model06_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat06_model4 = totalNsat(high_model06_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat06_model5 = totalNsat(high_model06_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat06_model6 = totalNsat(high_model06_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat06_model7 = totalNsat(high_model06_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat06_model8 = totalNsat(high_model06_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')

# models for 0.6 < 0.65

sumsat65_model1 = totalNsat(high_model65_1, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 1')
sumsat65_model2 = totalNsat(high_model65_2, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 2')
sumsat65_model3 = totalNsat(high_model65_3, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 3')
sumsat65_model4 = totalNsat(high_model65_4, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 4')
sumsat65_model5 = totalNsat(high_model65_5, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 5')
sumsat65_model6 = totalNsat(high_model65_6, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 6')
sumsat65_model7 = totalNsat(high_model65_7, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 7')
sumsat65_model8 = totalNsat(high_model65_8, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'model 8')

print('----')


# data

sumsat25_data = totalNsat(high_maskedNsat25, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat03_data = totalNsat(high_maskedNsat03, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat35_data = totalNsat(high_maskedNsat35, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat04_data = totalNsat(high_maskedNsat04, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat45_data = totalNsat(high_maskedNsat45, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat05_data = totalNsat(high_maskedNsat05, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat55_data = totalNsat(high_maskedNsat55, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat06_data = totalNsat(high_maskedNsat06, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')
sumsat65_data = totalNsat(high_maskedNsat65, 9.55, round(zrange[0], 2), round(zrange[1], 2), 'data')


mean number of satellites is 2.999929830858642
mean number of satellites is 0.9999298308586421
mean number of satellites is 3.999929830858642
mean number of satellites is 3.999929830858642
mean number of satellites is 4.999929830858642
mean number of satellites is 4.999929830858642
mean number of satellites is 6.999929830858641
mean number of satellites is 6.999929830858641
----
mean number of satellites is 3.0
mean number of satellites is 1.0
mean number of satellites is 4.0
mean number of satellites is 4.0
mean number of satellites is 5.0
mean number of satellites is 5.0
mean number of satellites is 7.0
mean number of satellites is 7.0
----
mean number of satellites is 3.0338003145308754
mean number of satellites is 1.0338003145308754
mean number of satellites is 4.033800314530875
mean number of satellites is 4.033800314530875
mean number of satellites is 5.033800314530875
mean number of satellites is 5.033800314530875
mean number of satellites is 7.033800314530875
mean number of sat

In [41]:
# perform k-sample Anderson-Darling test on model distributions and data

from scipy import stats

stat25_1, crit_vals25_1, sig_level25_1 = stats.anderson_ksamp([sumsat25_model1, sumsat25_data])
stat25_2, crit_vals25_2, sig_level25_2 = stats.anderson_ksamp([sumsat25_model2, sumsat25_data])
stat25_3, crit_vals25_3, sig_level25_3 = stats.anderson_ksamp([sumsat25_model3, sumsat25_data])
stat25_4, crit_vals25_4, sig_level25_4 = stats.anderson_ksamp([sumsat25_model4, sumsat25_data])
stat25_5, crit_vals25_5, sig_level25_5 = stats.anderson_ksamp([sumsat25_model5, sumsat25_data])
stat25_6, crit_vals25_6, sig_level25_6 = stats.anderson_ksamp([sumsat25_model6, sumsat25_data])
stat25_7, crit_vals25_7, sig_level25_7 = stats.anderson_ksamp([sumsat25_model7, sumsat25_data])
stat25_8, crit_vals25_8, sig_level25_8 = stats.anderson_ksamp([sumsat25_model8, sumsat25_data])

stat03_1, crit_vals03_1, sig_level03_1 = stats.anderson_ksamp([sumsat03_model1, sumsat03_data])
stat03_2, crit_vals03_2, sig_level03_2 = stats.anderson_ksamp([sumsat03_model2, sumsat03_data])
stat03_3, crit_vals03_3, sig_level03_3 = stats.anderson_ksamp([sumsat03_model3, sumsat03_data])
stat03_4, crit_vals03_4, sig_level03_4 = stats.anderson_ksamp([sumsat03_model4, sumsat03_data])
stat03_5, crit_vals03_5, sig_level03_5 = stats.anderson_ksamp([sumsat03_model5, sumsat03_data])
stat03_6, crit_vals03_6, sig_level03_6 = stats.anderson_ksamp([sumsat03_model6, sumsat03_data])
stat03_7, crit_vals03_7, sig_level03_7 = stats.anderson_ksamp([sumsat03_model7, sumsat03_data])
stat03_8, crit_vals03_8, sig_level03_8 = stats.anderson_ksamp([sumsat03_model8, sumsat03_data])

stat35_1, crit_vals35_1, sig_level35_1 = stats.anderson_ksamp([sumsat35_model1, sumsat35_data])
stat35_2, crit_vals35_2, sig_level35_2 = stats.anderson_ksamp([sumsat35_model2, sumsat35_data])
stat35_3, crit_vals35_3, sig_level35_3 = stats.anderson_ksamp([sumsat35_model3, sumsat35_data])
stat35_4, crit_vals35_4, sig_level35_4 = stats.anderson_ksamp([sumsat35_model4, sumsat35_data])
stat35_5, crit_vals35_5, sig_level35_5 = stats.anderson_ksamp([sumsat35_model5, sumsat35_data])
stat35_6, crit_vals35_6, sig_level35_6 = stats.anderson_ksamp([sumsat35_model6, sumsat35_data])
stat35_7, crit_vals35_7, sig_level35_7 = stats.anderson_ksamp([sumsat35_model7, sumsat35_data])
stat35_8, crit_vals35_8, sig_level35_8 = stats.anderson_ksamp([sumsat35_model8, sumsat35_data])

stat04_1, crit_vals04_1, sig_level04_1 = stats.anderson_ksamp([sumsat04_model1, sumsat04_data])
stat04_2, crit_vals04_2, sig_level04_2 = stats.anderson_ksamp([sumsat04_model2, sumsat04_data])
stat04_3, crit_vals04_3, sig_level04_3 = stats.anderson_ksamp([sumsat04_model3, sumsat04_data])
stat04_4, crit_vals04_4, sig_level04_4 = stats.anderson_ksamp([sumsat04_model4, sumsat04_data])
stat04_5, crit_vals04_5, sig_level04_5 = stats.anderson_ksamp([sumsat04_model5, sumsat04_data])
stat04_6, crit_vals04_6, sig_level04_6 = stats.anderson_ksamp([sumsat04_model6, sumsat04_data])
stat04_7, crit_vals04_7, sig_level04_7 = stats.anderson_ksamp([sumsat04_model7, sumsat04_data])
stat04_8, crit_vals04_8, sig_level04_8 = stats.anderson_ksamp([sumsat04_model8, sumsat04_data])

stat45_1, crit_vals45_1, sig_level45_1 = stats.anderson_ksamp([sumsat45_model1, sumsat45_data])
stat45_2, crit_vals45_2, sig_level45_2 = stats.anderson_ksamp([sumsat45_model2, sumsat45_data])
stat45_3, crit_vals45_3, sig_level45_3 = stats.anderson_ksamp([sumsat45_model3, sumsat45_data])
stat45_4, crit_vals45_4, sig_level45_4 = stats.anderson_ksamp([sumsat45_model4, sumsat45_data])
stat45_5, crit_vals45_5, sig_level45_5 = stats.anderson_ksamp([sumsat45_model5, sumsat45_data])
stat45_6, crit_vals45_6, sig_level45_6 = stats.anderson_ksamp([sumsat45_model6, sumsat45_data])
stat45_7, crit_vals45_7, sig_level45_7 = stats.anderson_ksamp([sumsat45_model7, sumsat45_data])
stat45_8, crit_vals45_8, sig_level45_8 = stats.anderson_ksamp([sumsat45_model8, sumsat45_data])

stat05_1, crit_vals05_1, sig_level05_1 = stats.anderson_ksamp([sumsat05_model1, sumsat05_data])
stat05_2, crit_vals05_2, sig_level05_2 = stats.anderson_ksamp([sumsat05_model2, sumsat05_data])
stat05_3, crit_vals05_3, sig_level05_3 = stats.anderson_ksamp([sumsat05_model3, sumsat05_data])
stat05_4, crit_vals05_4, sig_level05_4 = stats.anderson_ksamp([sumsat05_model4, sumsat05_data])
stat05_5, crit_vals05_5, sig_level05_5 = stats.anderson_ksamp([sumsat05_model5, sumsat05_data])
stat05_6, crit_vals05_6, sig_level05_6 = stats.anderson_ksamp([sumsat05_model6, sumsat05_data])
stat05_7, crit_vals05_7, sig_level05_7 = stats.anderson_ksamp([sumsat05_model7, sumsat05_data])
stat05_8, crit_vals05_8, sig_level05_8 = stats.anderson_ksamp([sumsat05_model8, sumsat05_data])

stat55_1, crit_vals55_1, sig_level55_1 = stats.anderson_ksamp([sumsat55_model1, sumsat55_data])
stat55_2, crit_vals55_2, sig_level55_2 = stats.anderson_ksamp([sumsat55_model2, sumsat55_data])
stat55_3, crit_vals55_3, sig_level55_3 = stats.anderson_ksamp([sumsat55_model3, sumsat55_data])
stat55_4, crit_vals55_4, sig_level55_4 = stats.anderson_ksamp([sumsat55_model4, sumsat55_data])
stat55_5, crit_vals55_5, sig_level55_5 = stats.anderson_ksamp([sumsat55_model5, sumsat55_data])
stat55_6, crit_vals55_6, sig_level55_6 = stats.anderson_ksamp([sumsat55_model6, sumsat55_data])
stat55_7, crit_vals55_7, sig_level55_7 = stats.anderson_ksamp([sumsat55_model7, sumsat55_data])
stat55_8, crit_vals55_8, sig_level55_8 = stats.anderson_ksamp([sumsat55_model8, sumsat55_data])

stat06_1, crit_vals06_1, sig_level06_1 = stats.anderson_ksamp([sumsat06_model1, sumsat06_data])
stat06_2, crit_vals06_2, sig_level06_2 = stats.anderson_ksamp([sumsat06_model2, sumsat06_data])
stat06_3, crit_vals06_3, sig_level06_3 = stats.anderson_ksamp([sumsat06_model3, sumsat06_data])
stat06_4, crit_vals06_4, sig_level06_4 = stats.anderson_ksamp([sumsat06_model4, sumsat06_data])
stat06_5, crit_vals06_5, sig_level06_5 = stats.anderson_ksamp([sumsat06_model5, sumsat06_data])
stat06_6, crit_vals06_6, sig_level06_6 = stats.anderson_ksamp([sumsat06_model6, sumsat06_data])
stat06_7, crit_vals06_7, sig_level06_7 = stats.anderson_ksamp([sumsat06_model7, sumsat06_data])
stat06_8, crit_vals06_8, sig_level06_8 = stats.anderson_ksamp([sumsat06_model8, sumsat06_data])

stat65_1, crit_vals65_1, sig_level65_1 = stats.anderson_ksamp([sumsat65_model1, sumsat06_data])
stat65_2, crit_vals65_2, sig_level65_2 = stats.anderson_ksamp([sumsat65_model2, sumsat06_data])
stat65_3, crit_vals65_3, sig_level65_3 = stats.anderson_ksamp([sumsat65_model3, sumsat06_data])
stat65_4, crit_vals65_4, sig_level65_4 = stats.anderson_ksamp([sumsat65_model4, sumsat06_data])
stat65_5, crit_vals65_5, sig_level65_5 = stats.anderson_ksamp([sumsat65_model5, sumsat06_data])
stat65_6, crit_vals65_6, sig_level65_6 = stats.anderson_ksamp([sumsat65_model6, sumsat06_data])
stat65_7, crit_vals65_7, sig_level65_7 = stats.anderson_ksamp([sumsat65_model7, sumsat06_data])
stat65_8, crit_vals65_8, sig_level65_8 = stats.anderson_ksamp([sumsat65_model8, sumsat06_data])

print('0.2 < 0.25')
print('----')
print('model 1')
print(stat25_1)
print(crit_vals25_1)
print(sig_level25_1)
print('----')

print('model 2')
print(stat25_2)
print(crit_vals25_2)
print(sig_level25_2)
print('----')

print('model 3')
print(stat25_3)
print(crit_vals25_3)
print(sig_level25_3)
print('----')

print('model 4')
print(stat25_4)
print(crit_vals25_4)
print(sig_level25_4)
print('----')

print('model 5')
print(stat25_5)
print(crit_vals25_5)
print(sig_level25_5)
print('----')

print('model 6')
print(stat25_6)
print(crit_vals25_6)
print(sig_level25_6)
print('----')

print('model 7')
print(stat25_7)
print(crit_vals25_7)
print(sig_level25_7)
print('----')

print('model 8')
print(stat25_8)
print(crit_vals25_8)
print(sig_level25_8)
print('----')
print()
print('----')
print('----')
print()
print('0.25 < 0.3')
print('----')
print('model 1')
print(stat03_1)
print(crit_vals03_1)
print(sig_level03_1)
print('----')

print('model 2')
print(stat03_2)
print(crit_vals03_2)
print(sig_level03_2)
print('----')

print('model 3')
print(stat03_3)
print(crit_vals03_3)
print(sig_level03_3)
print('----')

print('model 4')
print(stat03_4)
print(crit_vals03_4)
print(sig_level03_4)
print('----')

print('model 5')
print(stat03_5)
print(crit_vals03_5)
print(sig_level03_5)
print('----')

print('model 6')
print(stat03_6)
print(crit_vals03_6)
print(sig_level03_6)
print('----')

print('model 7')
print(stat03_7)
print(crit_vals03_7)
print(sig_level03_7)
print('----')

print('model 8')
print(stat03_8)
print(crit_vals03_8)
print(sig_level03_8)
print('----')
print()
print('----')
print('----')
print()
print('0.3 < 0.35')
print('----')
print('model 1')
print(stat35_1)
print(crit_vals35_1)
print(sig_level35_1)
print('----')

print('model 2')
print(stat35_2)
print(crit_vals35_2)
print(sig_level35_2)
print('----')

print('model 3')
print(stat35_3)
print(crit_vals35_3)
print(sig_level35_3)
print('----')

print('model 4')
print(stat35_4)
print(crit_vals35_4)
print(sig_level35_4)
print('----')

print('model 5')
print(stat35_5)
print(crit_vals35_5)
print(sig_level35_5)
print('----')

print('model 6')
print(stat35_6)
print(crit_vals35_6)
print(sig_level35_6)
print('----')

print('model 7')
print(stat35_7)
print(crit_vals35_7)
print(sig_level35_7)
print('----')

print('model 8')
print(stat35_8)
print(crit_vals35_8)
print(sig_level35_8)
print('----')
print()
print('----')
print('----')
print()
print('0.35 < 0.4')
print('----')
print('model 1')
print(stat04_1)
print(crit_vals04_1)
print(sig_level04_1)
print('----')

print('model 2')
print(stat04_2)
print(crit_vals04_2)
print(sig_level04_2)
print('----')

print('model 3')
print(stat04_3)
print(crit_vals04_3)
print(sig_level04_3)
print('----')

print('model 4')
print(stat04_4)
print(crit_vals04_4)
print(sig_level04_4)
print('----')

print('model 5')
print(stat04_5)
print(crit_vals04_5)
print(sig_level04_5)
print('----')

print('model 6')
print(stat04_6)
print(crit_vals04_6)
print(sig_level04_6)
print('----')

print('model 7')
print(stat04_7)
print(crit_vals04_7)
print(sig_level04_7)
print('----')

print('model 8')
print(stat04_8)
print(crit_vals04_8)
print(sig_level04_8)
print('----')
print()
print('----')
print('----')
print()
print('0.4 < 0.45')
print('----')
print('model 1')
print(stat45_1)
print(crit_vals45_1)
print(sig_level45_1)
print('----')

print('model 2')
print(stat45_2)
print(crit_vals45_2)
print(sig_level45_2)
print('----')

print('model 3')
print(stat45_3)
print(crit_vals45_3)
print(sig_level45_3)
print('----')

print('model 4')
print(stat45_4)
print(crit_vals45_4)
print(sig_level45_4)
print('----')

print('model 5')
print(stat45_5)
print(crit_vals45_5)
print(sig_level45_5)
print('----')

print('model 6')
print(stat45_6)
print(crit_vals45_6)
print(sig_level45_6)
print('----')

print('model 7')
print(stat45_7)
print(crit_vals45_7)
print(sig_level45_7)
print('----')

print('model 8')
print(stat45_8)
print(crit_vals45_8)
print(sig_level45_8)
print('----')
print()
print('----')
print('----')
print()
print('0.45 < 0.5')
print('----')
print('model 1')
print(stat05_1)
print(crit_vals05_1)
print(sig_level05_1)
print('----')

print('model 2')
print(stat05_2)
print(crit_vals05_2)
print(sig_level05_2)
print('----')

print('model 3')
print(stat05_3)
print(crit_vals05_3)
print(sig_level05_3)
print('----')

print('model 4')
print(stat05_4)
print(crit_vals05_4)
print(sig_level05_4)
print('----')

print('model 5')
print(stat05_5)
print(crit_vals05_5)
print(sig_level05_5)
print('----')

print('model 6')
print(stat05_6)
print(crit_vals05_6)
print(sig_level05_6)
print('----')

print('model 7')
print(stat05_7)
print(crit_vals05_7)
print(sig_level05_7)
print('----')

print('model 8')
print(stat05_8)
print(crit_vals05_8)
print(sig_level05_8)
print('----')
print()
print('----')
print('----')
print()
print('0.5 < 0.55')
print('----')
print('model 1')
print(stat55_1)
print(crit_vals55_1)
print(sig_level55_1)
print('----')

print('model 2')
print(stat55_2)
print(crit_vals55_2)
print(sig_level55_2)
print('----')

print('model 3')
print(stat55_3)
print(crit_vals55_3)
print(sig_level55_3)
print('----')

print('model 4')
print(stat55_4)
print(crit_vals55_4)
print(sig_level55_4)
print('----')

print('model 5')
print(stat55_5)
print(crit_vals55_5)
print(sig_level55_5)
print('----')

print('model 6')
print(stat55_6)
print(crit_vals55_6)
print(sig_level55_6)
print('----')

print('model 7')
print(stat55_7)
print(crit_vals55_7)
print(sig_level55_7)
print('----')

print('model 8')
print(stat55_8)
print(crit_vals55_8)
print(sig_level55_8)
print('----')
print()
print('----')
print('----')
print()
print('0.55 < 0.6')
print('----')
print('model 1')
print(stat06_1)
print(crit_vals06_1)
print(sig_level06_1)
print('----')

print('model 2')
print(stat06_2)
print(crit_vals06_2)
print(sig_level06_2)
print('----')

print('model 3')
print(stat06_3)
print(crit_vals06_3)
print(sig_level06_3)
print('----')

print('model 4')
print(stat06_4)
print(crit_vals06_4)
print(sig_level06_4)
print('----')

print('model 5')
print(stat06_5)
print(crit_vals06_5)
print(sig_level06_5)
print('----')

print('model 6')
print(stat06_6)
print(crit_vals06_6)
print(sig_level06_6)
print('----')

print('model 7')
print(stat06_7)
print(crit_vals06_7)
print(sig_level06_7)
print('----')

print('model 8')
print(stat06_8)
print(crit_vals06_8)
print(sig_level06_8)
print('----')
print()
print('----')
print('----')
print()
print('0.6 < 0.65')
print('----')
print('model 1')
print(stat65_1)
print(crit_vals65_1)
print(sig_level65_1)
print('----')

print('model 2')
print(stat65_2)
print(crit_vals65_2)
print(sig_level65_2)
print('----')

print('model 3')
print(stat65_3)
print(crit_vals65_3)
print(sig_level65_3)
print('----')

print('model 4')
print(stat65_4)
print(crit_vals65_4)
print(sig_level65_4)
print('----')

print('model 5')
print(stat65_5)
print(crit_vals65_5)
print(sig_level65_5)
print('----')

print('model 6')
print(stat65_6)
print(crit_vals65_6)
print(sig_level65_6)
print('----')

print('model 7')
print(stat65_7)
print(crit_vals65_7)
print(sig_level65_7)
print('----')

print('model 8')
print(stat65_8)
print(crit_vals65_8)
print(sig_level65_8)



0.2 < 0.25
----
model 1
245.6113225367999
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 2
245.6113225367999
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 3
245.6113225367999
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 4
245.6113225367999
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 5
245.6113225367999
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 6
245.6113225367999
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 7
245.6113225367999
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 8
245.6113225367999
[0.325 1.226 1.961 2.718 3.752]
inf
----

----
----

0.25 < 0.3
----
model 1
795.2047026202094
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 2
795.2047026202094
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 3
795.2047026202094
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 4
795.2047026202094
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 5
795.2047026202094
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 6
795.2047026202094
[0.325 1.226 1.961 2.718 3.752]
inf
----
model 7
795.204702

In [51]:
# perform k-sample K-S test on model distributions and data

from scipy import stats

stat25_1, pval25_1 = stats.ks_2samp(sumsat25_model1, sumsat25_data)
stat25_2, pval25_2 = stats.ks_2samp(sumsat25_model2, sumsat25_data)
stat25_3, pval25_3 = stats.ks_2samp(sumsat25_model3, sumsat25_data)
stat25_4, pval25_4 = stats.ks_2samp(sumsat25_model4, sumsat25_data)
stat25_5, pval25_5 = stats.ks_2samp(sumsat25_model5, sumsat25_data)
stat25_6, pval25_6 = stats.ks_2samp(sumsat25_model6, sumsat25_data)
stat25_7, pval25_7 = stats.ks_2samp(sumsat25_model7, sumsat25_data)
stat25_8, pval25_8 = stats.ks_2samp(sumsat25_model8, sumsat25_data)

stat03_1, pval03_1 = stats.ks_2samp(sumsat03_model1, sumsat03_data)
stat03_2, pval03_2 = stats.ks_2samp(sumsat03_model2, sumsat03_data)
stat03_3, pval03_3 = stats.ks_2samp(sumsat03_model3, sumsat03_data)
stat03_4, pval03_4 = stats.ks_2samp(sumsat03_model4, sumsat03_data)
stat03_5, pval03_5 = stats.ks_2samp(sumsat03_model5, sumsat03_data)
stat03_6, pval03_6 = stats.ks_2samp(sumsat03_model6, sumsat03_data)
stat03_7, pval03_7 = stats.ks_2samp(sumsat03_model7, sumsat03_data)
stat03_8, pval03_8 = stats.ks_2samp(sumsat03_model8, sumsat03_data)

stat35_1, pval35_1 = stats.ks_2samp(sumsat35_model1, sumsat35_data)
stat35_2, pval35_2 = stats.ks_2samp(sumsat35_model2, sumsat35_data)
stat35_3, pval35_3 = stats.ks_2samp(sumsat35_model3, sumsat35_data)
stat35_4, pval35_4 = stats.ks_2samp(sumsat35_model4, sumsat35_data)
stat35_5, pval35_5 = stats.ks_2samp(sumsat35_model5, sumsat35_data)
stat35_6, pval35_6 = stats.ks_2samp(sumsat35_model6, sumsat35_data)
stat35_7, pval35_7 = stats.ks_2samp(sumsat35_model7, sumsat35_data)
stat35_8, pval35_8 = stats.ks_2samp(sumsat35_model8, sumsat35_data)

stat04_1, pval04_1 = stats.ks_2samp(sumsat04_model1, sumsat04_data)
stat04_2, pval04_2 = stats.ks_2samp(sumsat04_model2, sumsat04_data)
stat04_3, pval04_3 = stats.ks_2samp(sumsat04_model3, sumsat04_data)
stat04_4, pval04_4 = stats.ks_2samp(sumsat04_model4, sumsat04_data)
stat04_5, pval04_5 = stats.ks_2samp(sumsat04_model5, sumsat04_data)
stat04_6, pval04_6 = stats.ks_2samp(sumsat04_model6, sumsat04_data)
stat04_7, pval04_7 = stats.ks_2samp(sumsat04_model7, sumsat04_data)
stat04_8, pval04_8 = stats.ks_2samp(sumsat04_model8, sumsat04_data)

stat45_1, pval45_1 = stats.ks_2samp(sumsat45_model1, sumsat45_data)
stat45_2, pval45_2 = stats.ks_2samp(sumsat45_model2, sumsat45_data)
stat45_3, pval45_3 = stats.ks_2samp(sumsat45_model3, sumsat45_data)
stat45_4, pval45_4 = stats.ks_2samp(sumsat45_model4, sumsat45_data)
stat45_5, pval45_5 = stats.ks_2samp(sumsat45_model5, sumsat45_data)
stat45_6, pval45_6 = stats.ks_2samp(sumsat45_model6, sumsat45_data)
stat45_7, pval45_7 = stats.ks_2samp(sumsat45_model7, sumsat45_data)
stat45_8, pval45_8 = stats.ks_2samp(sumsat45_model8, sumsat45_data)

stat05_1, pval05_1 = stats.ks_2samp(sumsat05_model1, sumsat05_data)
stat05_2, pval05_2 = stats.ks_2samp(sumsat05_model2, sumsat05_data)
stat05_3, pval05_3 = stats.ks_2samp(sumsat05_model3, sumsat05_data)
stat05_4, pval05_4 = stats.ks_2samp(sumsat05_model4, sumsat05_data)
stat05_5, pval05_5 = stats.ks_2samp(sumsat05_model5, sumsat05_data)
stat05_6, pval05_6 = stats.ks_2samp(sumsat05_model6, sumsat05_data)
stat05_7, pval05_7 = stats.ks_2samp(sumsat05_model7, sumsat05_data)
stat05_8, pval05_8 = stats.ks_2samp(sumsat05_model8, sumsat05_data)

stat55_1, pval55_1 = stats.ks_2samp(sumsat55_model1, sumsat55_data)
stat55_2, pval55_2 = stats.ks_2samp(sumsat55_model2, sumsat55_data)
stat55_3, pval55_3 = stats.ks_2samp(sumsat55_model3, sumsat55_data)
stat55_4, pval55_4 = stats.ks_2samp(sumsat55_model4, sumsat55_data)
stat55_5, pval55_5 = stats.ks_2samp(sumsat55_model5, sumsat55_data)
stat55_6, pval55_6 = stats.ks_2samp(sumsat55_model6, sumsat55_data)
stat55_7, pval55_7 = stats.ks_2samp(sumsat55_model7, sumsat55_data)
stat55_8, pval55_8 = stats.ks_2samp(sumsat55_model8, sumsat55_data)

stat06_1, pval06_1 = stats.ks_2samp(sumsat06_model1, sumsat06_data)
stat06_2, pval06_2 = stats.ks_2samp(sumsat06_model2, sumsat06_data)
stat06_3, pval06_3 = stats.ks_2samp(sumsat06_model3, sumsat06_data)
stat06_4, pval06_4 = stats.ks_2samp(sumsat06_model4, sumsat06_data)
stat06_5, pval06_5 = stats.ks_2samp(sumsat06_model5, sumsat06_data)
stat06_6, pval06_6 = stats.ks_2samp(sumsat06_model6, sumsat06_data)
stat06_7, pval06_7 = stats.ks_2samp(sumsat06_model7, sumsat06_data)
stat06_8, pval06_8 = stats.ks_2samp(sumsat06_model8, sumsat06_data)

stat65_1, pval65_1 = stats.ks_2samp(sumsat65_model1, sumsat06_data)
stat65_2, pval65_2 = stats.ks_2samp(sumsat65_model2, sumsat06_data)
stat65_3, pval65_3 = stats.ks_2samp(sumsat65_model3, sumsat06_data)
stat65_4, pval65_4 = stats.ks_2samp(sumsat65_model4, sumsat06_data)
stat65_5, pval65_5 = stats.ks_2samp(sumsat65_model5, sumsat06_data)
stat65_6, pval65_6 = stats.ks_2samp(sumsat65_model6, sumsat06_data)
stat65_7, pval65_7 = stats.ks_2samp(sumsat65_model7, sumsat06_data)
stat65_8, pval65_8 = stats.ks_2samp(sumsat65_model8, sumsat06_data)

print('0.2 < 0.25')
print('----')
print('model 1')
print(stat25_1)
print(pval25_1)
print('----')

print('model 2')
print(stat25_2)
print(pval25_2)
print('----')

print('model 3')
print(stat25_3)
print(pval25_3)
print('----')

print('model 4')
print(stat25_4)
print(pval25_4)
print('----')

print('model 5')
print(stat25_5)
print(pval25_5)
print('----')

print('model 6')
print(stat25_6)
print(pval25_6)
print('----')

print('model 7')
print(stat25_7)
print(pval25_7)
print('----')

print('model 8')
print(stat25_8)
print(pval25_8)
print('----')
print()
print('----')
print('----')
print()
print('0.25 < 0.3')
print('----')
print('model 1')
print(stat03_1)
print(pval03_1)
print('----')

print('model 2')
print(stat03_2)
print(pval03_2)
print('----')

print('model 3')
print(stat03_3)
print(pval03_3)
print('----')

print('model 4')
print(stat03_4)
print(pval03_4)
print('----')

print('model 5')
print(stat03_5)
print(pval03_5)
print('----')

print('model 6')
print(stat03_6)
print(pval03_6)
print('----')

print('model 7')
print(stat03_7)
print(pval03_7)
print('----')

print('model 8')
print(stat03_8)
print(pval03_8)
print('----')
print()
print('----')
print('----')
print()
print('0.3 < 0.35')
print('----')
print('model 1')
print(stat35_1)
print(pval35_1)
print('----')

print('model 2')
print(stat35_2)
print(pval35_2)
print('----')

print('model 3')
print(stat35_3)
print(pval35_3)
print('----')

print('model 4')
print(stat35_4)
print(pval35_4)
print('----')

print('model 5')
print(stat35_5)
print(pval35_5)
print('----')

print('model 6')
print(stat35_6)
print(pval35_6)
print('----')

print('model 7')
print(stat35_7)
print(pval35_7)
print('----')

print('model 8')
print(stat35_8)
print(pval35_8)
print('----')
print()
print('----')
print('----')
print()
print('0.35 < 0.4')
print('----')
print('model 1')
print(stat04_1)
print(pval04_1)
print('----')

print('model 2')
print(stat04_2)
print(pval04_2)
print('----')

print('model 3')
print(stat04_3)
print(pval04_3)
print('----')

print('model 4')
print(stat04_4)
print(pval04_4)
print('----')

print('model 5')
print(stat04_5)
print(pval04_5)
print('----')

print('model 6')
print(stat04_6)
print(pval04_6)
print('----')

print('model 7')
print(stat04_7)
print(pval04_7)
print('----')

print('model 8')
print(stat04_8)
print(pval04_8)
print('----')
print()
print('----')
print('----')
print()
print('0.4 < 0.45')
print('----')
print('model 1')
print(stat45_1)
print(pval45_1)
print('----')

print('model 2')
print(stat45_2)
print(pval45_2)
print('----')

print('model 3')
print(stat45_3)
print(pval45_3)
print('----')

print('model 4')
print(stat45_4)
print(pval45_4)
print('----')

print('model 5')
print(stat45_5)
print(pval45_5)
print('----')

print('model 6')
print(stat45_6)
print(pval45_6)
print('----')

print('model 7')
print(stat45_7)
print(pval45_7)
print('----')

print('model 8')
print(stat45_8)
print(pval45_8)
print('----')
print()
print('----')
print('----')
print()
print('0.45 < 0.5')
print('----')
print('model 1')
print(stat05_1)
print(pval05_1)
print('----')

print('model 2')
print(stat05_2)
print(pval05_2)
print('----')

print('model 3')
print(stat05_3)
print(pval05_3)
print('----')

print('model 4')
print(stat05_4)
print(pval05_4)
print('----')

print('model 5')
print(stat05_5)
print(pval05_5)
print('----')

print('model 6')
print(stat05_6)
print(pval05_6)
print('----')

print('model 7')
print(stat05_7)
print(pval05_7)
print('----')

print('model 8')
print(stat05_8)
print(pval05_8)
print('----')
print()
print('----')
print('----')
print()
print('0.5 < 0.55')
print('----')
print('model 1')
print(stat55_1)
print(pval55_1)
print('----')

print('model 2')
print(stat55_2)
print(pval55_2)
print('----')

print('model 3')
print(stat55_3)
print(pval55_3)
print('----')

print('model 4')
print(stat55_4)
print(pval55_4)
print('----')

print('model 5')
print(stat55_5)
print(pval55_5)
print('----')

print('model 6')
print(stat55_6)
print(pval55_6)
print('----')

print('model 7')
print(stat55_7)
print(pval55_7)
print('----')

print('model 8')
print(stat55_8)
print(pval55_8)
print('----')
print()
print('----')
print('----')
print()
print('0.55 < 0.6')
print('----')
print('model 1')
print(stat06_1)
print(pval06_1)
print('----')

print('model 2')
print(stat06_2)
print(pval06_2)
print('----')

print('model 3')
print(stat06_3)
print(pval06_3)
print('----')

print('model 4')
print(stat06_4)
print(pval06_4)
print('----')

print('model 5')
print(stat06_5)
print(pval06_5)
print('----')

print('model 6')
print(stat06_6)
print(pval06_6)
print('----')

print('model 7')
print(stat06_7)
print(pval06_7)
print('----')

print('model 8')
print(stat06_8)
print(pval06_8)
print('----')
print()
print('----')
print('----')
print()
print('0.6 < 0.65')
print('----')
print('model 1')
print(stat65_1)
print(pval65_1)
print('----')

print('model 2')
print(stat65_2)
print(pval65_2)
print('----')

print('model 3')
print(stat65_3)
print(pval65_3)
print('----')

print('model 4')
print(stat65_4)
print(pval65_4)
print('----')

print('model 5')
print(stat65_5)
print(pval65_5)
print('----')

print('model 6')
print(stat65_6)
print(pval65_6)
print('----')

print('model 7')
print(stat65_7)
print(pval65_7)
print('----')

print('model 8')
print(stat65_8)
print(pval65_8)



0.2 < 0.25
----
model 1
1.0
6.973059992644965e-71
----
model 2
1.0
6.973059992644965e-71
----
model 3
1.0
6.973059992644965e-71
----
model 4
1.0
6.973059992644965e-71
----
model 5
1.0
6.973059992644965e-71
----
model 6
1.0
6.973059992644965e-71
----
model 7
1.0
6.973059992644965e-71
----
model 8
1.0
6.973059992644965e-71
----

----
----

0.25 < 0.3
----
model 1
1.0
1.2926274129497469e-64
----
model 2
1.0
1.2926274129497469e-64
----
model 3
1.0
1.2926274129497469e-64
----
model 4
1.0
1.2926274129497469e-64
----
model 5
1.0
1.2926274129497469e-64
----
model 6
1.0
1.2926274129497469e-64
----
model 7
1.0
1.2926274129497469e-64
----
model 8
1.0
1.2926274129497469e-64
----

----
----

0.3 < 0.35
----
model 1
1.0
1.381606892900472e-97
----
model 2
0.875
8.17355108584608e-75
----
model 3
1.0
1.381606892900472e-97
----
model 4
1.0
1.381606892900472e-97
----
model 5
1.0
1.381606892900472e-97
----
model 6
1.0
1.381606892900472e-97
----
model 7
1.0
1.381606892900472e-97
----
model 8
1.0
1.38160689